In [1]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 37,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from sklearn.preprocessing import StandardScaler,MinMaxScaler\n",
    "from datetime import datetime"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 38,
   "metadata": {},
   "outputs": [],
   "source": [
    "#data=pd.read_csv(\"godavari_daily.csv\")\n",
    "data=pd.read_excel(\"data/Godavari.xlsx\")\n",
    "y=data['Flood']\n",
    "# data.drop('Flood',axis=1,inplace=True)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# dataset has benn updated to make a binary classification problem. "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 39,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "{0.0,\n",
       " 0.1,\n",
       " 0.2,\n",
       " 0.3,\n",
       " 0.4,\n",
       " 0.5,\n",
       " 0.6,\n",
       " 0.7,\n",
       " 0.8,\n",
       " 0.9,\n",
       " 1.0,\n",
       " 1.1,\n",
       " 1.2,\n",
       " 1.3,\n",
       " 1.4,\n",
       " 1.6,\n",
       " 1.7,\n",
       " 1.8,\n",
       " 1.9,\n",
       " 2.1,\n",
       " 2.2,\n",
       " 2.4,\n",
       " 2.6,\n",
       " 3.0,\n",
       " 3.1,\n",
       " 3.8,\n",
       " 4.0,\n",
       " 4.2,\n",
       " 4.8,\n",
       " 5.4,\n",
       " 5.5,\n",
       " 6.0,\n",
       " 6.1,\n",
       " 6.5,\n",
       " 6.9,\n",
       " 7.0,\n",
       " 7.1,\n",
       " 7.3,\n",
       " 7.4,\n",
       " 7.9,\n",
       " 8.4,\n",
       " 9.4,\n",
       " 9.8,\n",
       " 10.0}"
      ]
     },
     "execution_count": 39,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "set(y)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 40,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<matplotlib.axes._subplots.AxesSubplot at 0x24d92e3c860>"
      ]
     },
     "execution_count": 40,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAXQAAAD8CAYAAABn919SAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDMuMC4zLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvnQurowAAFWZJREFUeJzt3XuMXOV5x/Hf410bbHOxwQvlVowbZEGrCsgqQKlQy61AKVQqUk0TQlOIpTRpIW0VYaVJVFWNQhVRkjYhcYGUqATSOBQQ5RJuFrfWZs3VYIwNNmDA9toGfLf38vSPObO7Xrz27jnvnvOed74faTUzZ8/M+8zMmd+8856bubsAAPU3oeoCAABhEOgAkAgCHQASQaADQCIIdABIBIEOAIkg0AEgEQQ6ACSCQAeARLSX2diMGTN85syZZTYJALW3ZMmSDe7esb/5Sg30mTNnqqurq8wmAaD2zOzt0czHkAsAJIJAB4BEEOgAkAgCHQASQaADQCL2G+hmdpuZrTezpUOmHWZmj5jZiuxy+viWCQDYn9H00P9D0oXDpl0v6TF3P1HSY9ltAECF9hvo7v6kpE3DJl8m6fbs+u2S/jhwXUDLeXblBq3asK3qMlBjecfQj3T3DyQpuzxipBnNbK6ZdZlZV3d3d87mgPT92S2L9PvfXVh1GaixcV8p6u7z3b3T3Ts7Ova75yoAIKe8gb7OzI6SpOxyfbiSAAB55A30+yRdlV2/StK9YcoBAOQ1ms0W75T0v5Jmm9kaM7ta0ncknW9mKySdn90GAFRov0dbdPcrRvjXuYFrAQAUwJ6iAJAIAh0AEkGgA0AiCHQASASBDgCJINABIBEEOgAkgkAHgEQQ6ACQCAIdABJBoANAIgh0AEgEgQ4AiSDQASARBDoAJIJAB4BEEOgAkAgCHQASQaADQCIIdABIBIEOAIkg0IEIuHvVJSABBDoQAfIcIRDoQATIc4RAoAMRYMgFIRDoQAT6yXMEQKADEXAGXRAAgQ5EgBEXhECgA0AiCHQgAvTQEUKhQDezr5rZq2a21MzuNLMDQxUGtBLG0BFC7kA3s2Mk/bWkTnf/LUltkuaEKgwAMDZFh1zaJU02s3ZJUyS9X7wkoPUw5IIQcge6u78n6buS3pH0gaSP3f1XoQoDWgl5jhCKDLlMl3SZpBMkHS1pqpl9bi/zzTWzLjPr6u7uzl8pkDD2FEUIRYZczpO0yt273b1H0t2Sfmf4TO4+39073b2zo6OjQHNAuohzhFAk0N+RdIaZTTEzk3SupGVhygJaCx10hFBkDH2RpAWSnpf0SvZY8wPVBbQWAh0BtBe5s7t/S9K3AtUCtCy2Q0cI7CkKRIAhF4RAoAMRIM8RAoEORIDNFhECgQ5EgDhHCAQ6EAE66AiBQAciwFYuCIFAB2JAniMAAj0xu3r7qi4BOZDnCIFAT8izKzdo9t8/pMWrNlVdCsaIMXSEQKAn5MkVGyRJz60m0OuGMXSEQKAnpKevX5I0qY23tW7ooSMEPvkJ6c0CfWKbVVwJxoo8RwgEekJ29zVioZ0eeu2wpyhC4JOfEIZc6os8Rwh88hPSHHJpZ8gFaEkEekKanbwJRqDXDT10hECgAxFgs0WEQKADEaCHjhAI9IQQCvXFW4cQCPQEMYReP2y2iBAIdCACxDlCINATRGevfnjPEAKBDkSBREdxBHpCNm7bJYkx9Dqih44QCPSEPLNyY9UlIKdmnvNljCIIdCACzR46eY4iCHQgAs09RY0uOgog0IEIMIaOEAh0IAIMuSAEAh2IwOCQS8WFoNYIdCACgz10Eh35FQp0M5tmZgvM7HUzW2ZmZ4YqDGhJ5DkKaC94/+9JesjdLzezSZKmBKgJBbGlRP0who4Qcge6mR0i6WxJfy5J7r5b0u4wZQGthRNcIIQiQy6zJHVL+omZvWBmt5jZ1OEzmdlcM+sys67u7u4CzWG0OBRr/Qz00Omio4Aigd4u6TRJN7v7qZK2Sbp++EzuPt/dO929s6Ojo0BzQPpYKYoiigT6Gklr3H1RdnuBGgGPijGGXj/8pkIIuQPd3ddKetfMZmeTzpX0WpCqgBbTHCbjuxhFFN3K5a8k3ZFt4fKWpC8ULwloPQNHW6y0CtRdoUB39xcldQaqBWhZrMdGCOwpmiB6eXXE0RZRHIEORIAdixACgQ5EYGDEhURHAQQ6EAF66AiBQE8Qw7D1M7jZIm8e8iPQgQiwkQtCINCBCHAsF4RAoCeIbZrrZ+CMRRXXgXoj0IEYDPTQiXTkR6AniEyoH3b9RwgEOhABxtARAoEORIAzFiEEAh2IgLOrKAIg0BPEWW/qZ2AMnbcOBRDoQAQG9hStuA7UG4EORIAeOkIg0IEYDByci0RHfgR6gujl1Q9buSAEAh2IANuhIwQCHYgAx0NHCAQ6EIHBlaJEOvIj0IEIOIfIRAAEOhABNltECAQ6EAE66AiBQAei0DynaMVloNYI9ASRCfXj7FiEAAh0IAKMoSMEAh2IANuhIwQCHYjAwEmi6aKjAAIdiABbuSCEwoFuZm1m9oKZ3R+iIBRHJ69+OEk0QgjRQ79W0rIAjwO0LGcQHQEUCnQzO1bSH0q6JUw5QGsjz1FE0R76TZK+Jqk/QC1Ayxo8fC6RjvxyB7qZXSJpvbsv2c98c82sy8y6uru78zYHJG1gK5eK60C9FemhnyXpUjNbLekuSeeY2X8On8nd57t7p7t3dnR0FGgOSBdbuSCE3IHu7vPc/Vh3nylpjqTH3f1zwSoDWghnLEIIbIcORGBws0USHfm1h3gQd18oaWGIx0IIhELdNDdbpIeOIuihA0AiCHQgAqwTRQgEOhADEh0BEOgJYhy2fjjaIkIg0IEIcCgXhECgAxHgjEUIgUAHIsCORQiBQAciMHgsFxId+RHoQAQ4lgtCINCBCDCGjhAIdCAGzuFzURyBDkRgYMSFLjoKINATRCTUD9uhIwQCHYgAR1tECAQ6EAE2ckEIBDoQAYZcEAKBniAO8FQ/g5st8t4hPwIdiICz2SICINCBiNBBRxEEOhCBwTF0Eh35EehABJztXBAAgQ5EYODgXHTQUQCBDkSAPEcIBDoQAU5wgRAI9ASRCfXDCS4QAoEOoLCdPX1676MdVZfR8gh0IAJ1P2PR1bc/p7O+83jVZbQ8Ah2ISF3H0J9ZubHqEiACPUl1DYVWxuFzEQKBDkSAPUURAoEORICTRCOE3IFuZseZ2RNmtszMXjWza0MWVhevr92sbbt6qy4DNVf3laKIQ5Eeeq+kv3X3kySdIenLZnZymLLqoa/fdeFNT+mLP+2qupQ90MsDWlPuQHf3D9z9+ez6FknLJB0TqrA66M+6VYtXbaq4EtQdB+dCCEHG0M1spqRTJS3ay//mmlmXmXV1d3eHaC46sX0U+fleP4O7/tf755Wz8FWqcKCb2UGSfinpOnffPPz/7j7f3TvdvbOjo6Noc0CSODgXQigU6GY2UY0wv8Pd7w5TUn1wYl8Ek0jPNpGnUVvtee9ojd+Gt0pa5u43hiupfmJbhmv+q70l+bDLuqp7/bHZsrNH97zw3qjnL9JDP0vSlZLOMbMXs7+LCzxe7bAiC6HQs8XefLS9R9+499VRz5+7h+7uT6vFRxv4EAJ7aqwUbelYqBR7iiaCrQvqLZVfe2k8i3iM9WNNoCeI44HUzxvrtlZdAhJAoBcQU6c4plowdo+8tq7qEoJgOawWgR5ADMMd1VeAEGJYlopIZeiorgj0Alh4gT3V/PsoOmPNGAK9gJh2196jZ1d9OQAC2LB115jmJ9ADiOFncvUVAAjtylsXj2l+Ar2AaEM02sKQugj6NknZvrtvTPMT6AXE0DNviqgUABUh0AuIKUP3WHnCGDoqwoYC1SLQAQTDL8VqEegFxLTwxlQLWheLYbUI9CJYegFEJKlA7+933bzwTW3Z2VN1KaXbvGPwOecdQn9lzcd68JUPctfwj/e/pu89uiL3/Ze8/aEeTWQX+FYV04YCrSipQH902Trd8NDr+qf/WVZKezGtAPrmGI6ZPJI/+ren9aU7ns91X3fXrU+v0r88+kbu9v/k5md1zU+7ct8f1YvnE9Gakgr0HT2NbTa37uotpb2YOiMf76j2V0lff0QvRo3FtEyhfpIK9KYJJe+KH8NnsOpfC30kEcQXUtWiCvT+fteNj7yhTdt257r/4LFVAha1r/bKaWZUqv4gVd0+IsFyUKmoAv2plRv0/cdW6Bv3LM11/2Yvtaz+eUwrgKouhSEXoHpRBfqubAx8V+/Yjl/QFNPRD1sNQy6Qqh/6a3VRBXp/wUAeCPRA9ey3vZLaGY2hH6Qq6uqnhw5V/0ux1UUV6M0ompAzkQeWpbLG0CNaeKvOU4Zcwqh7D7fe1ddfVIHe19+4zLuVSnNMu/StXFiKGXIBIhBVoPc3AzlnF70ZKeUNucQTYlWvoCXPIVW/HMZo0Vsb9e9PvlVKW+2ltDJK/QV72M2FqbQOesmbSe5L1SMeDLlAYshlb/50/v9Jkr549qxxbyuqHnrzyz3vGHqTlXxAcDolaQT6/Cff1JNvdFddRiXcXfPuflmX/OtT2r67nD2tEV5Ugd7soeeN42amTCjpWfWV/YtgH6qO0/4EvtW+/cDr+vxtYzuHYyp29PTpzsXvaul7m/WzRe/kfpwEFoNaiyrQi25H3uwllrVStLev0d7Esr5B9qXiT1IKPfQYVPU2hnr7Ylqv1IoiSKJBRReFsrdyGfgCieBVrDpPU+iht7JgX8gsBpWKIIoGNQP5tfc36+9+8dKYd1ZpLpRtRQfhR6m32V4EYy5V94yam5yinviFlYZCgW5mF5rZcjNbaWbXhypq+botWrBkjdZu3jmm+/WVvNXJYA+9+kCvOlDpoddbb3+YBYilYGRl7E2dO9DNrE3SDyRdJOlkSVeY2clFihn+dMfaayh7yKUnS9GyfhHsS1+gD2T+9uv9Ua57/UU11wcVxff6yMrY+a5ID/0zkla6+1vuvlvSXZIuC1NWw1g/ZGUPuTTba48g0EN9IPOqew+9p8KfODEcB6fVv9DKUMZrbHn37DKzyyVd6O7XZLevlHS6u39lpPscctxsP/2r80d8zA+379aGrYPHQj/+8Cma1Db675y1H+/Ull29aptgmjVj6qjvl9eOnj6t+XCHJOnEIw4a9/b2ZcX6rQPXjz70QE09YOz7jDUfI89z2dnbp3c3FXstirRfVJ+73ureVkn7Lmll9twnT2zTsdMnl9q+1PhCW71x+8Dtsb4GzffuhBlTo+jgxKT52vxGx9Qxjx407/v2DZcscffO/c1fZE/RvVX2iW8HM5sraa4kHXL0LJ145L4XlAdeWatPHz9dS97+UL959CFjKuhTRxykB5eu1QUnH1naOPqaD3forE8drkMnTyynwREcM32yFi5v7BRzyq9Py/UYvf2utzdu2+97NJKBQM95/+27+7R5Z0/u+xe1asM2zT7yYM3qGP/OwHBbd/ZqguV/70JoBvp5Jx2hSe1j+/E+ZVKbNmzdrZOOOng8Squ1CWZ6Z9N2zf61sb82h0yeqJfXfDTq+YsE+hpJxw25fayk94fP5O7zJc2XpM7OTv/hZz9doEkAaD327dHNV2QM/TlJJ5rZCWY2SdIcSfcVeDwAQAG5e+ju3mtmX5H0sKQ2Sbe5+6vBKgMAjEmhoy26+wOSHghUCwCggKj2FAUA5EegA0AiCHQASASBDgCJINABIBG5d/3P1ZjZFknLS2uwmBmSNlRdxBhQ7/ipU60S9Y6nqmo93t079jdT2SeJXj6a4xHEwMy66lKrRL3jqU61StQ7nmKvlSEXAEgEgQ4AiSg70Ec+dm586lSrRL3jqU61StQ7nqKutdSVogCA8cOQCwAkopRAH6+TSeeo4zYzW29mS4dMO8zMHjGzFdnl9Gy6mdn3s5pfNrPThtznqmz+FWZ21TjVepyZPWFmy8zsVTO7NvJ6DzSzxWb2UlbvP2TTTzCzRVnbP88OtSwzOyC7vTL7/8whjzUvm77czP5gPOrN2mkzsxfM7P4a1LrazF4xsxfNrCubFuWykLUzzcwWmNnr2TJ8Zqz1mtns7HVt/m02s+tirXef3H1c/9Q4tO6bkmZJmiTpJUknj3e7I9RytqTTJC0dMu2fJV2fXb9e0g3Z9YslPajGmZnOkLQom36YpLeyy+nZ9enjUOtRkk7Lrh8s6Q01TsYda70m6aDs+kRJi7I6/kvSnGz6jyR9Kbv+l5J+lF2fI+nn2fWTs2XkAEknZMtO2zgtD38j6WeS7s9ux1zrakkzhk2LclnI2rpd0jXZ9UmSpsVc75C62yStlXR8Her9RP3j3oB0pqSHh9yeJ2lemU9yWD0ztWegL5d0VHb9KDW2lZekH0u6Yvh8kq6Q9OMh0/eYbxzrvlfS+XWoV9IUSc9LOl2NnTDahy8LahxH/8zsens2nw1fPobOF7jGYyU9JukcSfdnbUdZa/bYq/XJQI9yWZB0iKRVytbRxV7vsBovkPRMXeod/lfGkMsxkt4dcntNNi0WR7r7B5KUXR6RTR+p7tKfT/YT/1Q1er3R1psNYbwoab2kR9TosX7k7r17aXugruz/H0s6vMR6b5L0NUn92e3DI65Vapyv91dmtsQa5+mV4l0WZknqlvSTbEjrFjObGnG9Q82RdGd2vQ717qGMQB/VyaQjNFLdpT4fMztI0i8lXefum/c1616mlVqvu/e5+ylq9H4/I+mkfbRdWb1mdomk9e6+ZOjkfbRb+Wsr6Sx3P03SRZK+bGZn72PequttV2No82Z3P1XSNjWGLEZSdb2NIhrrTC6V9Iv9zbqXaaXXuzdlBPqoTiZdoXVmdpQkZZfrs+kj1V3a8zGziWqE+R3ufnfs9Ta5+0eSFqoxvjjNzJqHmBja9kBd2f8PlbSppHrPknSpma2WdJcawy43RVqrJMnd388u10v6bzW+MGNdFtZIWuPui7LbC9QI+FjrbbpI0vPuvi67HXu9n1BGoMd+Mun7JDXXRl+lxlh1c/rnszXaZ0j6OPvZ9bCkC8xserbW+4JsWlBmZpJulbTM3W+sQb0dZjYtuz5Z0nmSlkl6QtLlI9TbfB6XS3rcGwOP90mak21ZcoKkEyUtDlmru89z92PdfaYay+Pj7v7ZGGuVJDObamYHN6+r8R4uVaTLgruvlfSumc3OJp0r6bVY6x3iCg0OtzTrirneTypjoF6NtcJvqDGm+vUyVxIMq+NOSR9I6lHj2/RqNcZCH5O0Irs8LJvXJP0gq/kVSZ1DHucvJK3M/r4wTrX+rho/116W9GL2d3HE9f62pBeyepdK+mY2fZYaIbdSjZ+yB2TTD8xur8z+P2vIY309ex7LJV00zsvE72lwK5coa83qein7e7X5GYp1WcjaOUVSV7Y83KPGVh8x1ztF0kZJhw6ZFm29I/2xpygAJII9RQEgEQQ6ACSCQAeARBDoAJAIAh0AEkGgA0AiCHQASASBDgCJ+H8U5AApPqzqwgAAAABJRU5ErkJggg==\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "y.plot(x = data['Date'], y = y)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 41,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "(7670, 6)\n"
     ]
    },
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Date</th>\n",
       "      <th>Discharge</th>\n",
       "      <th>flood runoff</th>\n",
       "      <th>daily runoff</th>\n",
       "      <th>weekly runoff</th>\n",
       "      <th>Flood</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>1998-01-01</td>\n",
       "      <td>3244.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.91</td>\n",
       "      <td>NaN</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>1998-01-02</td>\n",
       "      <td>1591.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.45</td>\n",
       "      <td>NaN</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>1998-01-03</td>\n",
       "      <td>1973.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.55</td>\n",
       "      <td>NaN</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>1998-01-04</td>\n",
       "      <td>1973.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.55</td>\n",
       "      <td>NaN</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>1998-01-05</td>\n",
       "      <td>2011.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.56</td>\n",
       "      <td>NaN</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "        Date  Discharge  flood runoff  daily runoff  weekly runoff  Flood\n",
       "0 1998-01-01     3244.0           0.0          0.91            NaN    0.0\n",
       "1 1998-01-02     1591.0           0.0          0.45            NaN    0.0\n",
       "2 1998-01-03     1973.0           0.0          0.55            NaN    0.0\n",
       "3 1998-01-04     1973.0           0.0          0.55            NaN    0.0\n",
       "4 1998-01-05     2011.0           0.0          0.56            NaN    0.0"
      ]
     },
     "execution_count": 41,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "print(data.shape)\n",
    "data.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 42,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "C:\\Users\\user\\Anaconda3\\lib\\site-packages\\pandas\\core\\indexing.py:190: SettingWithCopyWarning: \n",
      "A value is trying to be set on a copy of a slice from a DataFrame\n",
      "\n",
      "See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy\n",
      "  self._setitem_with_indexer(indexer, value)\n"
     ]
    }
   ],
   "source": [
    "for i in range(len(y)):\n",
    "    if y.iloc[i] >= 0.5 :\n",
    "        y.iloc[i]=1.0\n",
    "    else :\n",
    "        y.iloc[i]=0.0\n",
    "data.drop('Flood',axis=1,inplace=True)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 43,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.0    7599\n",
       "1.0      71\n",
       "Name: Flood, dtype: int64"
      ]
     },
     "execution_count": 43,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "y.value_counts()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 44,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Date</th>\n",
       "      <th>Discharge</th>\n",
       "      <th>flood runoff</th>\n",
       "      <th>daily runoff</th>\n",
       "      <th>weekly runoff</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>1998-01-01</td>\n",
       "      <td>3244.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.91</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>1998-01-02</td>\n",
       "      <td>1591.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.45</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>1998-01-03</td>\n",
       "      <td>1973.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.55</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>1998-01-04</td>\n",
       "      <td>1973.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.55</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>1998-01-05</td>\n",
       "      <td>2011.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.56</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "        Date  Discharge  flood runoff  daily runoff  weekly runoff\n",
       "0 1998-01-01     3244.0           0.0          0.91            NaN\n",
       "1 1998-01-02     1591.0           0.0          0.45            NaN\n",
       "2 1998-01-03     1973.0           0.0          0.55            NaN\n",
       "3 1998-01-04     1973.0           0.0          0.55            NaN\n",
       "4 1998-01-05     2011.0           0.0          0.56            NaN"
      ]
     },
     "execution_count": 44,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "data.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 45,
   "metadata": {},
   "outputs": [],
   "source": [
    "d1=pd.DataFrame()\n",
    "d1[\"Day\"]=data['Date']\n",
    "d1['Months']=data['Date']\n",
    "d1['Year']=data['Date']\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 46,
   "metadata": {},
   "outputs": [],
   "source": [
    "data['Date']=pd.to_datetime(data['Date'])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 47,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Day</th>\n",
       "      <th>Months</th>\n",
       "      <th>Year</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>2</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>3</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>4</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>5</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "   Day  Months  Year\n",
       "0    1       1  1998\n",
       "1    2       1  1998\n",
       "2    3       1  1998\n",
       "3    4       1  1998\n",
       "4    5       1  1998"
      ]
     },
     "execution_count": 47,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "d1[\"Year\"]=data.Date.dt.year\n",
    "d1[\"Months\"]=data.Date.dt.month\n",
    "d1[\"Day\"]=data.Date.dt.day\n",
    "d1.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 48,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "numpy.int64"
      ]
     },
     "execution_count": 48,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "type(d1['Day'][0])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 49,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Date</th>\n",
       "      <th>Discharge</th>\n",
       "      <th>flood runoff</th>\n",
       "      <th>daily runoff</th>\n",
       "      <th>weekly runoff</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>1998-01-01</td>\n",
       "      <td>3244.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.91</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>1998-01-02</td>\n",
       "      <td>1591.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.45</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>1998-01-03</td>\n",
       "      <td>1973.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.55</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>1998-01-04</td>\n",
       "      <td>1973.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.55</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>1998-01-05</td>\n",
       "      <td>2011.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.56</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "        Date  Discharge  flood runoff  daily runoff  weekly runoff\n",
       "0 1998-01-01     3244.0           0.0          0.91            NaN\n",
       "1 1998-01-02     1591.0           0.0          0.45            NaN\n",
       "2 1998-01-03     1973.0           0.0          0.55            NaN\n",
       "3 1998-01-04     1973.0           0.0          0.55            NaN\n",
       "4 1998-01-05     2011.0           0.0          0.56            NaN"
      ]
     },
     "execution_count": 49,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "data.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 50,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "<class 'pandas.core.frame.DataFrame'>\n",
      "RangeIndex: 7670 entries, 0 to 7669\n",
      "Data columns (total 5 columns):\n",
      "Date             7670 non-null datetime64[ns]\n",
      "Discharge        7670 non-null float64\n",
      "flood runoff     7670 non-null float64\n",
      "daily runoff     7670 non-null float64\n",
      "weekly runoff    7670 non-null float64\n",
      "dtypes: datetime64[ns](1), float64(4)\n",
      "memory usage: 299.7 KB\n"
     ]
    }
   ],
   "source": [
    "data = data.fillna(data.mean())\n",
    "data.info()\n",
    "data.drop(columns='Date',axis=1,inplace=True)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 51,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Discharge</th>\n",
       "      <th>flood runoff</th>\n",
       "      <th>daily runoff</th>\n",
       "      <th>weekly runoff</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>3244.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.91</td>\n",
       "      <td>7.464144</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>1591.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.45</td>\n",
       "      <td>7.464144</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>1973.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.55</td>\n",
       "      <td>7.464144</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>1973.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.55</td>\n",
       "      <td>7.464144</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>2011.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.56</td>\n",
       "      <td>7.464144</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "   Discharge  flood runoff  daily runoff  weekly runoff\n",
       "0     3244.0           0.0          0.91       7.464144\n",
       "1     1591.0           0.0          0.45       7.464144\n",
       "2     1973.0           0.0          0.55       7.464144\n",
       "3     1973.0           0.0          0.55       7.464144\n",
       "4     2011.0           0.0          0.56       7.464144"
      ]
     },
     "execution_count": 51,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "data.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 52,
   "metadata": {
    "scrolled": true
   },
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Day</th>\n",
       "      <th>Months</th>\n",
       "      <th>Year</th>\n",
       "      <th>Discharge</th>\n",
       "      <th>flood runoff</th>\n",
       "      <th>daily runoff</th>\n",
       "      <th>weekly runoff</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "      <td>3244.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.91</td>\n",
       "      <td>7.464144</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>2</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "      <td>1591.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.45</td>\n",
       "      <td>7.464144</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>3</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "      <td>1973.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.55</td>\n",
       "      <td>7.464144</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>4</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "      <td>1973.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.55</td>\n",
       "      <td>7.464144</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>5</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "      <td>2011.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.56</td>\n",
       "      <td>7.464144</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "   Day  Months  Year  Discharge  flood runoff  daily runoff  weekly runoff\n",
       "0    1       1  1998     3244.0           0.0          0.91       7.464144\n",
       "1    2       1  1998     1591.0           0.0          0.45       7.464144\n",
       "2    3       1  1998     1973.0           0.0          0.55       7.464144\n",
       "3    4       1  1998     1973.0           0.0          0.55       7.464144\n",
       "4    5       1  1998     2011.0           0.0          0.56       7.464144"
      ]
     },
     "execution_count": 52,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "data=pd.concat([d1,data],axis=1)\n",
    "data.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": 53,
   "metadata": {},
   "outputs": [],
   "source": [
    "scl=MinMaxScaler()\n",
    "data1=scl.fit_transform(data.iloc[:,3:])\n",
    "data1=scl.fit_transform(data1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 54,
   "metadata": {},
   "outputs": [],
   "source": [
    "column=['Discharge','flood runoff','daily runoff','weekly runoff']"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 55,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Discharge</th>\n",
       "      <th>flood runoff</th>\n",
       "      <th>daily runoff</th>\n",
       "      <th>weekly runoff</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>0.107581</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.107820</td>\n",
       "      <td>0.155503</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>0.052762</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.053318</td>\n",
       "      <td>0.155503</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>0.065431</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.065166</td>\n",
       "      <td>0.155503</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>0.065431</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.065166</td>\n",
       "      <td>0.155503</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>0.066691</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.066351</td>\n",
       "      <td>0.155503</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "   Discharge  flood runoff  daily runoff  weekly runoff\n",
       "0   0.107581           0.0      0.107820       0.155503\n",
       "1   0.052762           0.0      0.053318       0.155503\n",
       "2   0.065431           0.0      0.065166       0.155503\n",
       "3   0.065431           0.0      0.065166       0.155503\n",
       "4   0.066691           0.0      0.066351       0.155503"
      ]
     },
     "execution_count": 55,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "data1=pd.DataFrame(data1,columns=column)\n",
    "data1.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 56,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Discharge</th>\n",
       "      <th>flood runoff</th>\n",
       "      <th>daily runoff</th>\n",
       "      <th>weekly runoff</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>0.107581</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.107820</td>\n",
       "      <td>0.155503</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>0.052762</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.053318</td>\n",
       "      <td>0.155503</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>0.065431</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.065166</td>\n",
       "      <td>0.155503</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>0.065431</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.065166</td>\n",
       "      <td>0.155503</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>0.066691</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.066351</td>\n",
       "      <td>0.155503</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "   Discharge  flood runoff  daily runoff  weekly runoff\n",
       "0   0.107581           0.0      0.107820       0.155503\n",
       "1   0.052762           0.0      0.053318       0.155503\n",
       "2   0.065431           0.0      0.065166       0.155503\n",
       "3   0.065431           0.0      0.065166       0.155503\n",
       "4   0.066691           0.0      0.066351       0.155503"
      ]
     },
     "execution_count": 56,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "data1.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 57,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Discharge</th>\n",
       "      <th>flood runoff</th>\n",
       "      <th>daily runoff</th>\n",
       "      <th>weekly runoff</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>count</th>\n",
       "      <td>7670.000000</td>\n",
       "      <td>7670.000000</td>\n",
       "      <td>7670.000000</td>\n",
       "      <td>7670.000000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>mean</th>\n",
       "      <td>0.126924</td>\n",
       "      <td>0.002907</td>\n",
       "      <td>0.126465</td>\n",
       "      <td>0.155503</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>std</th>\n",
       "      <td>0.112001</td>\n",
       "      <td>0.039334</td>\n",
       "      <td>0.112190</td>\n",
       "      <td>0.130361</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>min</th>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>25%</th>\n",
       "      <td>0.052762</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.052133</td>\n",
       "      <td>0.068542</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>50%</th>\n",
       "      <td>0.093404</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.092417</td>\n",
       "      <td>0.115000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>75%</th>\n",
       "      <td>0.162557</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.162322</td>\n",
       "      <td>0.199583</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>max</th>\n",
       "      <td>1.000000</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>1.000000</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "         Discharge  flood runoff  daily runoff  weekly runoff\n",
       "count  7670.000000   7670.000000   7670.000000    7670.000000\n",
       "mean      0.126924      0.002907      0.126465       0.155503\n",
       "std       0.112001      0.039334      0.112190       0.130361\n",
       "min       0.000000      0.000000      0.000000       0.000000\n",
       "25%       0.052762      0.000000      0.052133       0.068542\n",
       "50%       0.093404      0.000000      0.092417       0.115000\n",
       "75%       0.162557      0.000000      0.162322       0.199583\n",
       "max       1.000000      1.000000      1.000000       1.000000"
      ]
     },
     "execution_count": 57,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "data1.describe()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 58,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Day</th>\n",
       "      <th>Months</th>\n",
       "      <th>Year</th>\n",
       "      <th>Discharge</th>\n",
       "      <th>flood runoff</th>\n",
       "      <th>daily runoff</th>\n",
       "      <th>weekly runoff</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>count</th>\n",
       "      <td>7670.000000</td>\n",
       "      <td>7670.000000</td>\n",
       "      <td>7670.000000</td>\n",
       "      <td>7670.000000</td>\n",
       "      <td>7670.000000</td>\n",
       "      <td>7670.000000</td>\n",
       "      <td>7670.000000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>mean</th>\n",
       "      <td>15.729205</td>\n",
       "      <td>6.523077</td>\n",
       "      <td>2008.000000</td>\n",
       "      <td>3827.261216</td>\n",
       "      <td>0.101752</td>\n",
       "      <td>1.067362</td>\n",
       "      <td>7.464144</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>std</th>\n",
       "      <td>8.800483</td>\n",
       "      <td>3.448888</td>\n",
       "      <td>6.055444</td>\n",
       "      <td>3377.267298</td>\n",
       "      <td>1.376689</td>\n",
       "      <td>0.946885</td>\n",
       "      <td>6.257307</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>min</th>\n",
       "      <td>1.000000</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>1998.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>25%</th>\n",
       "      <td>8.000000</td>\n",
       "      <td>4.000000</td>\n",
       "      <td>2003.000000</td>\n",
       "      <td>1591.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.440000</td>\n",
       "      <td>3.290000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>50%</th>\n",
       "      <td>16.000000</td>\n",
       "      <td>7.000000</td>\n",
       "      <td>2008.000000</td>\n",
       "      <td>2816.500000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.780000</td>\n",
       "      <td>5.520000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>75%</th>\n",
       "      <td>23.000000</td>\n",
       "      <td>10.000000</td>\n",
       "      <td>2013.000000</td>\n",
       "      <td>4901.750000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>1.370000</td>\n",
       "      <td>9.580000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>max</th>\n",
       "      <td>31.000000</td>\n",
       "      <td>12.000000</td>\n",
       "      <td>2018.000000</td>\n",
       "      <td>30154.000000</td>\n",
       "      <td>35.000000</td>\n",
       "      <td>8.440000</td>\n",
       "      <td>48.000000</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "               Day       Months         Year     Discharge  flood runoff  \\\n",
       "count  7670.000000  7670.000000  7670.000000   7670.000000   7670.000000   \n",
       "mean     15.729205     6.523077  2008.000000   3827.261216      0.101752   \n",
       "std       8.800483     3.448888     6.055444   3377.267298      1.376689   \n",
       "min       1.000000     1.000000  1998.000000      0.000000      0.000000   \n",
       "25%       8.000000     4.000000  2003.000000   1591.000000      0.000000   \n",
       "50%      16.000000     7.000000  2008.000000   2816.500000      0.000000   \n",
       "75%      23.000000    10.000000  2013.000000   4901.750000      0.000000   \n",
       "max      31.000000    12.000000  2018.000000  30154.000000     35.000000   \n",
       "\n",
       "       daily runoff  weekly runoff  \n",
       "count   7670.000000    7670.000000  \n",
       "mean       1.067362       7.464144  \n",
       "std        0.946885       6.257307  \n",
       "min        0.000000       0.000000  \n",
       "25%        0.440000       3.290000  \n",
       "50%        0.780000       5.520000  \n",
       "75%        1.370000       9.580000  \n",
       "max        8.440000      48.000000  "
      ]
     },
     "execution_count": 58,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "data.describe()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 59,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Day</th>\n",
       "      <th>Months</th>\n",
       "      <th>Year</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>2</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>3</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>4</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>5</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "   Day  Months  Year\n",
       "0    1       1  1998\n",
       "1    2       1  1998\n",
       "2    3       1  1998\n",
       "3    4       1  1998\n",
       "4    5       1  1998"
      ]
     },
     "execution_count": 59,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "x=pd.DataFrame(data=data['Day'])\n",
    "x['Months']=data['Months']\n",
    "x['Year']=data['Year']\n",
    "x.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 60,
   "metadata": {},
   "outputs": [],
   "source": [
    "data1=pd.concat([x,data1],axis=1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 61,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Day</th>\n",
       "      <th>Months</th>\n",
       "      <th>Year</th>\n",
       "      <th>Discharge</th>\n",
       "      <th>flood runoff</th>\n",
       "      <th>daily runoff</th>\n",
       "      <th>weekly runoff</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "      <td>0.107581</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.107820</td>\n",
       "      <td>0.155503</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>2</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "      <td>0.052762</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.053318</td>\n",
       "      <td>0.155503</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>3</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "      <td>0.065431</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.065166</td>\n",
       "      <td>0.155503</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>4</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "      <td>0.065431</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.065166</td>\n",
       "      <td>0.155503</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>5</td>\n",
       "      <td>1</td>\n",
       "      <td>1998</td>\n",
       "      <td>0.066691</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.066351</td>\n",
       "      <td>0.155503</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "   Day  Months  Year  Discharge  flood runoff  daily runoff  weekly runoff\n",
       "0    1       1  1998   0.107581           0.0      0.107820       0.155503\n",
       "1    2       1  1998   0.052762           0.0      0.053318       0.155503\n",
       "2    3       1  1998   0.065431           0.0      0.065166       0.155503\n",
       "3    4       1  1998   0.065431           0.0      0.065166       0.155503\n",
       "4    5       1  1998   0.066691           0.0      0.066351       0.155503"
      ]
     },
     "execution_count": 61,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "data=data1\n",
    "data.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 62,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 864x864 with 9 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "data.hist(figsize=(12,12));"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 63,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Index(['Day', 'Months', 'Year', 'Discharge', 'flood runoff', 'daily runoff',\n",
      "       'weekly runoff'],\n",
      "      dtype='object')\n"
     ]
    }
   ],
   "source": [
    "print(data.columns)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 64,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 936x576 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "data[['Year', 'flood runoff', 'daily runoff', 'weekly runoff']].groupby(\"Year\").sum().plot(figsize=(13,8));"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 65,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "5479\n"
     ]
    }
   ],
   "source": [
    "locate=0;\n",
    "for i in range(len(data[\"Day\"])):\n",
    "    if(data[\"Day\"][i]==31 and data[\"Months\"][i]==12 and data[\"Year\"][i]==2012):\n",
    "        locate=i;\n",
    "        break;\n",
    "\n",
    "i=locate+1\n",
    "print(i)\n",
    "\n",
    "x_train=data.iloc[0:i,:]\n",
    "y_train=y.iloc[0:i]\n",
    "x_test=data.iloc[i:,:]\n",
    "y_test=y.iloc[i:]\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 66,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "(5479, 7)\n",
      "(5479,)\n",
      "(2191, 7)\n",
      "(2191,)\n"
     ]
    }
   ],
   "source": [
    "print(x_train.shape)\n",
    "print(y_train.shape)\n",
    "print(x_test.shape)\n",
    "print(y_test.shape)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 67,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.0    2176\n",
       "1.0      15\n",
       "Name: Flood, dtype: int64"
      ]
     },
     "execution_count": 67,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "y_test.value_counts()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 68,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.model_selection import train_test_split"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": 69,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "0.6256020324551246\n",
      "0.43633856467607424\n"
     ]
    }
   ],
   "source": [
    "from sklearn.linear_model import LinearRegression\n",
    "\n",
    "reg=LinearRegression()\n",
    "reg.fit(x_train,y_train)\n",
    "yp=reg.predict(x_test)\n",
    "print(reg.score(x_train,y_train))\n",
    "print(reg.score(x_test,y_test))\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 71,
   "metadata": {},
   "outputs": [],
   "source": [
    "# from sklearn import linear_model\n",
    "# from sklearn.metrics import mean_absolute_error\n",
    "# # linear model\n",
    "# reg = linear_model.ElasticNet(alpha=0.5)\n",
    "# reg.fit(X_train_res_shuf, Y_train_res_shuf)\n",
    "# y_pred = reg.predict(x_test)\n",
    "# print(mean_absolute_error(y_test, y_pred))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 72,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "2191"
      ]
     },
     "execution_count": 72,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "yt=np.array(y_test)\n",
    "len((y_test))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 74,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Y_pre[\"ypre\"].value_counts()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 75,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Y_pre=pd.DataFrame()\n",
    "# Y_pre[\"ypre\"]=y_pred\n",
    "# Y_pre[\"ytest\"]=yt\n",
    "# #Y_pre"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 76,
   "metadata": {
    "scrolled": true
   },
   "outputs": [],
   "source": [
    "# Y_pre[['ypre','ytest']].plot(figsize=(19,8));"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 77,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Linear regression will not work for binary classification  pr"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# SMOTE Oversampling\n",
    "#!pip install imblearn"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 78,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Number transactions X_train dataset:  (5369, 7)\n",
      "Number transactions Y_train dataset:  (5369,)\n",
      "Number transactions X_test dataset:  (2301, 7)\n",
      "Number transactions Y_test dataset:  (2301,)\n"
     ]
    }
   ],
   "source": [
    "from sklearn.model_selection import train_test_split\n",
    "\n",
    "X_train, X_test, Y_train, Y_test = train_test_split(data, y, test_size=0.3, random_state=0)\n",
    "\n",
    "print(\"Number transactions X_train dataset: \", X_train.shape)\n",
    "print(\"Number transactions Y_train dataset: \", Y_train.shape)\n",
    "print(\"Number transactions X_test dataset: \", X_test.shape)\n",
    "print(\"Number transactions Y_test dataset: \", Y_test.shape)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 79,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Before OverSampling, counts of label '1': 46\n",
      "Before OverSampling, counts of label '0': 5323 \n",
      "\n",
      "After OverSampling, the shape of train_X: (10646, 7)\n",
      "After OverSampling, the shape of train_y: (10646,) \n",
      "\n",
      "After OverSampling, counts of label '1': 5323\n",
      "After OverSampling, counts of label '0': 5323\n"
     ]
    }
   ],
   "source": [
    "from imblearn.over_sampling import SMOTE\n",
    "\n",
    "print(\"Before OverSampling, counts of label '1': {}\".format(sum(Y_train==1)))\n",
    "print(\"Before OverSampling, counts of label '0': {} \\n\".format(sum(Y_train==0)))\n",
    "\n",
    "sm = SMOTE(random_state=2)\n",
    "X_train_res, Y_train_res = sm.fit_sample(X_train, Y_train.ravel())\n",
    "\n",
    "print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))\n",
    "print('After OverSampling, the shape of train_y: {} \\n'.format(Y_train_res.shape))\n",
    "\n",
    "print(\"After OverSampling, counts of label '1': {}\".format(sum(Y_train_res==1)))\n",
    "print(\"After OverSampling, counts of label '0': {}\".format(sum(Y_train_res==0)))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 80,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<matplotlib.axes._subplots.AxesSubplot at 0x24d94f15a90>"
      ]
     },
     "execution_count": 80,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAXcAAAD8CAYAAACMwORRAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDMuMC4zLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvnQurowAAIABJREFUeJztnXmcFMX5/9+1Fwss933pLogcigIiiAhC8BYxGo2awyRqvGL0qzH5ocYDo4Z4RGNCNEbRaFS8ohIEkVNF5VjCDQss93Ituxy7C+xdvz+mZ3aunumZ6bmf9+u1r+3prq56qnvmU9VPPV2ltNYIgiAIqUVGvA0QBEEQ7EfEXRAEIQURcRcEQUhBRNwFQRBSEBF3QRCEFETEXRAEIQURcRcEQUhBRNwFQRBSEBF3QRCEFCQrXgV37NhR5+fnx6t4QRCEpGTFihVlWutOwdLFTdzz8/MpLCyMV/GCIAhJiVJqp5V04pYRBEFIQUTcBUEQUhARd0EQhBRExF0QBCEFEXEXBEFIQYKKu1JqmlKqVCm1zuS4Ukq9qJQqVkqtUUoNtd9MQRAEIRSs9NzfAC4JcPxSoK/xdyvwUuRmCYIgCJEQNM5da/2VUio/QJIrgTe1Y72+JUqptkqpblrrfRFbt2k2dDsTWneH0iI4Xg75o0LLY91HcGQXnHevY/vwThh9H2gNa96DARMhp0VT+pIVsPMb6H85dOjTtH/3csjOha6DfMtY+g/IaQlDftK0b/tXUF0Bzdt52lyyAjKzHPXatQSatYIupzmOffk0dB4AuW0hrzPMvBeatYaLnoDKfVBfDRmZ0Od7nuWv/RD6XgS5rf1fg4Z6WDMdzvwRZLi159VHYctcGHSN//NOHIHlr0JDLQz8PnQZCAfWQ00lnHQOnDgMWxdAj2GwbxXUVcOZ1znO1RpWvQOn/8Bx3dxxHht0LWTl+Ja75gM4VgrDb4Nd30Kr7g6796+Fhrome/eudOTVw8/DYkOd4/6613n1e4772izPf33dKSmEzXOCpxOEBMWOl5h6ALvdPpcY+3zEXSl1K47ePSeddFLwnN+9Hlr3hPvWw99HOPY9djQ06z68yfG//xVN230vdAjbx7fBzm9h4otN6V81hHPuw55lvXaB//K1htm/c2y7i/u/rmjadj/Hmf9jR2HaxU3bR3bBwif912HzbM/P7vntXwcf3exopK57y//53/0N5j0KuhGG3ti0/9O7YOMMR4PibGDc+fj2prK//JOj3JfObbLho1ugeJ7nOe0LoNdw2Pw5fHonlG6Ai73qteETx7FDW2H8I57HyorhP7c4trWGLx7ytctp7ytjfa+Hk8XPO65nRrajwdm1FD6+FQb/GL7/d7+XyYNFU6B4LqCCpxWEBMQOcff37fe76rbW+hXgFYBhw4ZZW5m7oiRswzyoP9G0XXfC0fsEqDpgT/6RUl8T3nl1xx3/KwM8KB0vc/w/cdhzf8UeI48T+CXYtT/q53htleN/tSG4xw76pgl0zP0+nTjkv1wze91x5l19xLDLuN+BrpM7ugF6ng23zAueVhBiyWRrHQ47omVKgF5un3sCe23IVxAEQQgTO8R9BnCjETVzDnDUFn+7IAiCEDZWQiHfBb4D+imlSpRSNyulbldK3W4kmQVsA4qBfwJ3Rs1aQUhQ1u05yuFjtTErb8+RE2itaWjU1Dc0xqxcq9Q1NPLpqj044iygrKqGiuo6lm4rZ2f5Mcv5HDleS229Z/1KDh+nsdGaV9cutNauuiQLQcVda32D1rqb1jpba91Ta/2a1vplrfXLxnGttf6V1rqP1nqQ1lqmehTSiqqaeib8dTFD/jCXe99bRV2UxXbT/kpGTVnAa4u3M/KP8zlz8hemafcdPcHEvy2mrCrMMR23fI7X1pse/7a4jPxJn7H5gGNsY+rCYu6ZvorZ6/YDMOyJeYyasoDrXlnC+c8s8jh3w94KbnhlCd9tLffJd/Djc7nz7f+5Pu8oO8Z5f1rI3xYWh1yHmWv2snhLWcjnARQ8MIsbpy0L61x/HDpWy1tLLE3uGDbyhqog+MNPL+14bT0rdh7iwY/XMv65RYBD2E9/tClk8uOVe1i+4xANAXqW+49WM2N107DUxn0VbC87xt4jFgaKgV2HHIPoS7aVU1pZw7HaBoY9MReA+oZG7vj3CtaWOAatX/9mB2tKjjJ92S7e/G4H1778LY2Nmgf+s4Z1e6xFnr0wbzMj/7iAG15Z4tq3bs9Rj57sZ2sdntil2x2D4AcqHI3J4eNNTzOV1b6NQ2V1HZe9+DXfbSvnV+/8z+c4wLyNTUEPzmvk3RBU1zWwoyzwE8Fd76zkJ68tDZhGa83vPlzNkm2+Dc3XYTYM/rhn+koe/mQdRfsrPPYX7a9gi9FARkrc5nMXhESnrqGRRRsOcOHALgDc995qPl+/3yNNxYk6n/N+9M+ljDm1E2/eNByA0spqGhuha5tc4/gStpUd48IBXWiek8mlf/nade6OKZcHtcsZK+HefpRV1fLF+v306ZzH7HX72bS/kgX3j0UZiZ/9YrMrbWllDe8u282ColKWPnhB0PJemLcFgNVGgzFn/X5ue2sFz157JlXVdQzoZvJ+hQWq60J7yjFrMu/49woWbjrItqcuIyPDUelDx2r5estBrhzcI6Qy3i8s4f3CEr767TgOH6/lzF5tQzrfCocMF15dvWeNLnnB8V2w8j0IhvTcBcGEzQeq+OWbhS6f9rq9vj1dM7H5anNTmOfwJ+dzzh/nuz7vO1ptnOt7dnVdAydqGwLa5RRsbx/wrW+t8E1rc5x+/qTP+GSlI4R2y4FKHvvvBq5z69F7Ey03tfKq1pfG9S471uR+uv3fK7hn+ipLT0RbD1b59PzHPLOQK6d+Q0W1bwPuzc7yYwHdVuB42hny+BeUV9W4rkuD1tz0xnI27K0IeG44iLgLggk1xkBeIH36bE3oUb8Nxi+7tMLXDz78yXkMeOTzgOdnGMrmz67ZhntkW9kxfvTPJT4iaAdOP3ogrJbrr4ELmD5I8uFPNjWi+41GNNgYyInaBsY/9yVjn13kv0y307/dWsaWA5U+Qn7+M4v4+evLA5bz8pdbOXy8jrveWUmjUZGl28pZUFTKZS9+HfDccEgbcT8Y4YCSkL44BcWfsGwttR75AXCwssYV/eHujnFS4ccv7c0OI9pk6Tbfl7y+cvMLf7u1nAyLIltb30hVTfCy3fGns9MWb+fsJ6P34pezMQjUeGitQ4qm2X34eMDjH/2v6WW9Z+Zs4sLnv+I2P09Jy7abvHTntMv4/52bPz8aja+TtBH3ihOhfXGFMEmycDErNNpYpxI3ITlRF9j98ss3C/l01R6Pfct3HGLyfzeYnu8tMFbdMjdOW+oxMGwFf6GB28uOcbCyqSP1r2938OEK62+ZNzQGDjl0HgpUr99+uIbeD86yXGaw2/v4zA2ubWej4W/ANXg5vgXZ7TZzJ23E3eMOmm0nM1bqYZbG9NxgXzwLX8xAdvk9poIcD5KnaVrl9Tnoia5THp+5gYrqOvZ4+W5X7z7Ce4W7/ZzbxOfrwnuf73htPXM3HOCe6asAh5gs3lLGXxeEFgL49RbfKR6cvd8DFTUu//kSr6eA4tKqoL1f90tZ3+A/7ZbSKu7/YLXfY0dP1Hk0BHUNjfR5cBaPz9zAn+du5uFPmmYZX1tylFMfms03xY4nE+8er7up3o2Jc0DYyd8XFbOmxDEtxe5Dxy351F0E6Wo7o6S2HqxyPaFtOVBJ/qTPmLW2yZ3ldBW5Z1daUR0w7/KqGh78eK1lU9M4WibRJoQK1x4r55mlieY1CJR3LK595GU4c3hn6S7eWbrL5/iVU78JeP7LX25lyuwi1+fCHYdcYh2Mx/+7wePz9QEGLQPhjHBx579uYZj/994qD4E59aHZ1Lr5qK1GbXg3cv6ulzfe8fnHjYHk17/Z4ZP2ubmbqG1o5B9fbQMc/vRrXvqWW0YXcO4pHf3m76zXxyv38Px1g137n/58E0+ziR1TLmf00ws9zvndh2sC2rx6t6NRqGvQLN5SxgMfr2H3oaZGf+n2ck7pnMf4574E4It7x3DR81/55LP1oMO1dqym6elr+FPzfdIB3PzGcpZsK2dAt9YU7jzsN40/0ljchUTkQGUNXeJtBA7/aIOOrIFwF3aAa17+zifN8h3+/bRbSqsiKjsQT83ytMu9wan1Gnz8ttg8tvvVxdvDtiF/0mc++wK9G1DsdT2c1yeQ2O0sb3KB+XOJeL/5CvBBCC4kfzHzK3Ycpkvrpimub/fjm3fn+Xmb/e5/v3A3f5pdxLKHLmB+USkQuK7+EHEXEoqq6vqEEHdnpEW0+ZnJW48r3H7If52/xW+aWPCjVwO/9BMrSg5be8HLjG+KfX3kp/5+tp+UkfHc3M08N9e/YIeC8wli1JQFYeeRPj53N8pjOAeIkJz4e5syGhwPEtMO2CIW6c5N/wocphgttgV5azYY+4P44QORluLuPogjCELq488Fk+qktLjHeuY4IXU4fFw6AEJyk9LiftTPvB+CYIW6NOzpCalFSou7IESCPPcJyYyIuyAIQgoi4p4gVNWKCwmiO9eGIKQTIu4JwqGUCc8UZ4YgJAIi7pGSKnPTCB7IA4SQ7Ii4CzYTmSza1VY6J2aql3BYIU0RcRdiSm19I7PXB1/sIVJWG7P+rdrteI3/iBEWW1Zl3f2lpf8uJDFpKe4yaBc/Pl5ZwqJNvtPQ2o2zx+6cjMq5qPS+o5HNUSIIyUJairsQP2pN5v0WBMFe0kbcpbeeHMh9EgR7SBtxl6AWQRDSibQRd0EIBSXx+kKSI+IuCCZItIyQzIi4C2mBv2XWBCGVSQpxv+4fvmtPphqp00cMLKJxr2fcDRCE2JAU4r50u/9FhIXUw+5oGemvC+mKJXFXSl2ilNqklCpWSk3yc/wkpdRCpdRKpdQapdRl9psqxJxgShuFuEW7vCe+lkmXXUgvgoq7UioTmApcCgwEblBKDfRK9nvgfa31EOB64O92GxopntEPZtvJjJV6mKWJ5jUIlHewciO31zvqxarES1MgJDtWeu7DgWKt9TatdS0wHbjSK40GWhvbbYC99pkYJRLsbZmwIzOs1MMsTTSvQYC8Y9GcSqSLkO5kWUjTA9jt9rkEGOGV5jHgC6XUr4GWwAW2WCcIgiCEhZWeu78ukHfn6wbgDa11T+Ay4C2llE/eSqlblVKFSqnCgwejP3mUIDhJFeebIFjFiriXAL3cPvfE1+1yM/A+gNb6OyAX6Oidkdb6Fa31MK31sE6dOoVnsZDUJJg3zC+7jRkkBSGZsSLuy4G+SqkCpVQOjgHTGV5pdgHjAZRSA3CIu3TN48yBimq+KS6LtxkhkQjiP/rphfE2QRAiJqjPXWtdr5S6C5gDZALTtNbrlVKPA4Va6xnAb4B/KqXuxfEE/HMtrwTGnQl/XczByhp2jI+3JcmHUvL1FZIbKwOqaK1nAbO89j3itr0BGGWvaUKkHKysCXhcawn5E4RUJSneUBWiw54jjlWJDlRU+z1eXdcYS3MSDgmnFJIZEfc0pqqmHoAjJ/yvK7r7cBgDi+LNEISEIH3EXfndFAIS7EpFYfqBaN2dRBipFYQYkj7iLgiCkEaIuEdMKvshwp37JXyitgJSiMFbshKTkOyIuAv2It4PQUgI0kfcpSMmhIjW0lIJyUv6iLsgCEIaIeIuCIKQgoi4JwDr9hyNtwkxQ4lTXhBigoh7AjDhr4vjbULKIjMcCemKiHuCoOQlGwO5DoJgB2kp7qKjsaWxUfPH2RspNZnDxpPE6WonjiWCEDqWZoVMNeRRPYr4ubZLtpfzjy+3sXFfJZec1jX2NglCGpKWPXfBGsHnebH2CORsTOsbgs8yqaP2WCWPa0J6IeIumJJKcqjFySKkGWkk7m4/bm2yncxYqYdZmmheAq8yPTrmQaeuMbM3FIO9yrdYWZlbRkh20kjcBUEQ0oc0FvfEcjqE/wBhpR5macK/BnM3HAizzMC+fPeXnF77ZnuoZlkq3yqyEpOQzKSxuAuR8Ms3C6NeRm19+Mv81TWm9xKBgpA24i59sPRiy4GqeJsgCHElbcTdnbeW7Ii3CT7IG6qCINhJWor7xn2Vru3GNIiKKK2sAeBgVU2cLUkeJFpGSHbSUtwBth50PLZvP3jMNE1DY/AfuE6CUMpio67bAtRVEITUIm3FvaK6DoBjtQ2mafo8OCtW5lCWMr3qxG/sAL4tLmPqwuKAaZKjJoLgn7QV90Tjwf+sjUk5ByqqyZ/0GR8U7g6aNhxx2334eBhnxZ4fvbqUZ+ZsAmDK7CJmrN4bZ4sEwV5SWty1yXYicrzO/AnCTpzuqI/+VxIw3eFjtQSNMfIzCHz0eF24pjkzDeusn762lPxJn4WV78tfbuXud1eGVa4gJCopLe6CNbyHDRYWlTLkD3M5YVODE4thia+3lEW/EEFIIkTcBR8Kdx6ylO7oidB66RLtKQixI3HFPQmiUBwki52hEywcsLzKyuIbTXxTXG7B6RKd6xlqVJO0Q0Kyk7jiHgHVdQ1M+mgNR47XRr2sbWUSXuiBH1VMpB57KKbI3DJCMmNJ3JVSlyilNimlipVSk0zS/FAptUEptV4p9Y69ZobGB4W7mb58N8/N3Wyaxq6f7UXPf2VTTgJIj1kQ7CLoMntKqUxgKnAhUAIsV0rN0FpvcEvTF3gAGKW1PqyU6hwtg62gfTaC88B/1nDp6d0YEw2DEokoe5GSxpsmCCmOlZ77cKBYa71Na10LTAeu9ErzS2Cq1vowgNa61F4zo8+7y3Zz47Rl8TYjZijpIwtCSmNF3HsA7m+8lBj73DkVOFUp9Y1SaolS6hJ/GSmlblVKFSqlCg8ePBiexUnInz4vorI60vjv9EA6/oJgD1bE3V8Xz/s3mAX0BcYCNwCvKqXa+pyk9Sta62Fa62GdOnUK1dak5aVFW3nuC3P/v5B4yMRhQrJjRdxLgF5un3sC3u9qlwCfaq3rtNbbgU04xD7xiZGTuCaChScC0WjYX9sQIP8QqxhNl00iRc4EQ6JlhGTGirgvB/oqpQqUUjnA9cAMrzSfAOMAlFIdcbhptoVj0Mpdh8mf9Bmrdx8J53QPAq14n0wiE4gtpY7pi3cdsn9Ol2DiFh3xi86NEaEW0o2g4q61rgfuAuYAG4H3tdbrlVKPK6UmGsnmAOVKqQ3AQuC3WuvycAxaUOQYi/1yc3R98skc1VHuNoOkc1bigPVJIl1LlUZXEOJN0FBIAK31LGCW175H3LY1cJ/xl5CYakYSqsmuQ8fpkNfMY5/VtiqJ2zRBEEIgJd9QTSeSsG0SBCEGpJG4N/VZ3fUwdlER0S3HWj3MPM/a65OdtgbKK3A55u1WCPaF6X+TaBkh2Ym7uC8sKnWtimQ3gf3QidXlDX/Az8p5JpIezUvgdn2t6Gt1XQPDn5zHd9vCGqrxQQZQhXQnruJ+oKKaX7yxnLveaVooIdYDnaGWd/qjc6LWGCUKUQ2FNMl739FqSitrmLZ4h63lzd1wwGMRD5F8IV2Im7jX1jcy4qn5AGwvq/I5bvYjnLN+v+UyZq+zntYqVTX1FO2rjDif0srQpstNFcxcPhv3VcTYEkFIbeIm7ofcpuMNpfccirhHi1DnBvfHx//bE/a5ys3lkSo90XeX7bKctr4xOi+ECUIqEXefO8TGFWNnGck81Obf9igP9trcAr29ZKe9GQpCCpIQ4h49orWqT2zOsUSYyqlUlHr9MWipN+yP3C0WjFR5IhLSl4QQ98ZQBCGZu802obXmWE19vM1IWNbvPRpvEwQh7sRN3N17Rsk2FUA4ceDeHexIXBXvF+7mtEfnsD3KS/wFr2XolQgWiWPHV+HyFxf7ybep3PkbD1jKR8IphWQmbuKuPbaTTt3jytwNjvl3tpb6Rhn5I9QB4Fnr9oVsU6SoEFo775TPB1hO0R+T/7sheCJBSHLiJu4HK30nv7KEBQ3YVR58hsRPV3nPWuyfKj/ujyRrilxYlc9tB5Nr0e+/zN8SbxMsM/HM7jEtr2VOZkzLExKHhPC52xFa6M6ri7cHTWPVpfFh4e7giWKO/6kUzNhpNHbuV3nFzsNU1TTYaxbhP4XZ/R0IWFbSNs+hkz41jYxJl/aPtwm2Y2lWyGjjv+du8rUM4duqvJw/nvu1nzTWcNehcOcg8fY9h5KPpw5qr/++HD1RC1mgtHadW1PfiMrxPXeTWyRK8IYjgM3OggKIdk1dPaOemMvgXm1dbhnPMv2fa23CBc+6eX+2fr69xHrWi2Qbz4oXLZslhBTaSkL03A8dq2XQo3N4Yd7msHtVT8zcwE9fW2p6/N73VoVrni1E40fmIRQ25X+8NrIonFCmLli1+whlVbXM21ga0tBloLSfrgr+cpgIXuz54bCeAY//YlR+bAxJIxJC3AEqa+p5YV6T7zTUHs6ri7fz9ZYy0+MNbj9oRxSEcts2Z6WfFaGi8VgfSmSG57VRXv/N89bKu1/se27ENXOW4fa/us6/+8f9ic3/4ixW5rD05J7pq4w0ZtfF+nWORrRMusbfPHnVoIDHrzkrsPgLoZMw4m6ZGP86/A28xrvnV1ZVGzyRCfGYDPPmfxW6GRD78sHznpndv31HT8TGGCFiMmLwPXrv1nOiX0gUSThx33XI8QOrrDZxD8gjtYvVu4/YLtbxbrisEK3fdbTfGwDo17V11MtIROy+Z9v+eLnNOfqSnRVdefztxf0CHn/i+6dHlH/Cift/Vzt6yq98Hdb62lHhc6+470TRvxcXFLN6t/W3Md3DT6OFd9SLmUsmEmobEuUOhM5tY3rH24SU48KBXaKSb7QfDs7p3SGq+SecuNuJXTfntx+s8fhsFrZXWlHN0eOxneu9zFgs24rcbdhbQVGU52XZe8TzHYMvNlh7GzQUdpbHIg4/Og1IhgV/wuVndGPtYxcFTffTc04mJzPwTzhRwj5DeUktVMJ92rz+7F72GuKHnMwM3r9tpM/+aT8fRrMATwY5WRkRP5Untbg/M6eIAQ9/HvNyzb5Lw5+az/Cn5vk99tXmgzz08droGWXQ+4HP+N2Hq02PH6iwPo98ONMPTF++mze/28HhY1bGBXzPt/I7DemlN/OiYsq7vzyH139xtqW02RmKVrnZAdM8fc0Z/OH7pzP5ytM89ndrkxu2jfEimHsiWtwax6eo7/XvwmndW/O41/1zsvmJSyMuI6nFferCrZyoa2DPkaaBsEOWRCVy3lm6i6L9vgtM1NQ3eqz842Tv0WreXto0Z/nfFxVHxa5GDe8XlkQlb6s88ul6pi6MTv3Cx7+619abzw1v1ob07tQy5NJH9unAuH6dAegeRQGec+8Yj8/RXFXLLn417hTX9oBukY1JPHPNGZGa4yI3O/S3e3t39Pxu/OX6waZplVLcODI/5DKsktTi7mTUlAWu7UA/Vn8cOlbLy19uDa1ADQ9+vJZLXvg6tPPcOGyT+8b9p7s4QChoNKgJcK2rIoyXtx//Un3P9JV+95uRm51By5zIXngJ9uARjgujoGNLvrh3DK2D9PhDZf5vzg+a5i43cTbDrEbNwxBQJ5cP6sZZJ7fzcF9cOyw8V0ufjr4NdjgNzWWDurm2n/7BGVzq9tmM1342zO/+sJ5Q3UgJcY+UKbOLQkqfKH5M8BSKzQfC96fvO3rC1miRRpNvZiL0I93HTNyXYgzWy5088TSWPDA+anZ5s2OKZ0RIC7d5YpyWOgflnrzqdE7t0gpwNEB24ey9BnL33G/RrfL178Z5XL/3bj2HBfc7Gg8zv/kvRxeY5jf1x0P56I5zLZU9oqA9ADef55vfqV3yOPeUjn7PO71HaALvnv+1xotbLYLM7zMsv73f/T3bNQ+pbG9E3C1Q6TV5WCKFC67285JVIKb7W85Ow8g/LmDcs4uwayCxus5/rz5el2793oqIy+/etjltW+SEfN7VQ3t4fA72/TFrYs7t0xRd4ezdF3RsyY4pl3NuH//iFElH5JNfjQqrMX73l+ewwKvHrxT0at+Crm6NxIjeHejWxlPAvMt76PKBQcuz8nvs3tZRzmndW1Pg1kuf+evz/A54/sEIQ5zxq/PY9tRlAfPOzmyyul3Lpu+H8x6d3qNNYONM7B/XrzOf/mpU4HMDkHTiHuw+flMcW9dEvHEfb7BChdn7A0nGphCjfj5f74jaOXyslr1Hw1uc3OlCMPOabHj8Ytf2Sz8e6tr+8w/N/a6hEE6nYvyA8MMEB/dqa7nszq2aubZP69HaR7StEsvGX+MQXn8Ndq4RyZKRoYJGOF1nU9RN61xfd9+ZbvcgVJJO3IPxmw+aIkXcJ39yvz2e+xOoGx4B1uphlqZpf6AfcUMYTsBAdgW32cy1o7n4ha8sWuCZh9X77e/nPOqUwHHJLdx88VZ8rd6E8tal1aQXDOjMf+70dV3cNMrc3eFRjsWCZt59XsDzgo0jxPLNaUsTz4VgUCI9ybuTdOJeuPMQr9rwglMk84ZE42aGm6WVegSX9CYC+d37PjTLmlEBynQci/4v2Y5b5G5nn04tTWauDCdf/9ZlREnhhp7UzuNz69wsHrkiuLvDHW+bT+mc5/G5c6vIIoCi8ZsKNlmZXYRi+u3n9/FzvrUcWvnp2Qci6cR996ETPPHZxrjakKANtS389sM1psciHb0XAhOr3muBn8gQM5yDzO7iu+A35/P5PaMBOM9kIDJc7LwET19zpmvb2Rh1bW1/GGooaxFMurS/z0C5k2BPC/27tgrJrqSdxNhfLLnQRE29/a/920EiRMvEG8fc4b5TQTiEVJtepFGndGR+kWOJxUA64B714607034+jMG92mEVf+X07uQQym1PXRbVBmnGXdYGEws6tgia5vbz+zDs5HaM6N2BW0b35sGP19LFJqFPxKcWsNhzV0pdopTapJQqVkpNCpDuGqWUVkr5D9xMEWK5alC4/POrxJmbJ1m4cdpS6htCe08iHP71i+E84Gflnx+NOAmAC0wGQe2Y8/x7/bvQvmXoET/+vvEZGcqjt5lnLHihgCwbpm3sZ9JT7eBl/28vDr6KUmaGYoQRNvqjESexY8rlLnv9EYr13dvm8sHtI5nzf2NM09wxtg8jTeaSyTKibfp6ubr1Dmt4AAAZTklEQVS8GV7gP2TSjKDirpTKBKYClwIDgRuUUj4OO6VUK+BuwHzFjBQhEaXd26Znvwht0Wi7CPajqEvgSb/qGrS16CM3QZs80f/r44Ho1b4Ft53fh5m/Po/fXHgqAAO7tebRKway6YlLPF6EcR8MdRfSs01iowH+/MMmd0SfTuaCUfSHS1jz2EVcPbQHM399HlcP6cHlXgPBzhJD7c9kZWaw5clLmXX3aH53SfA4+DbNHS9fDezeFFeeafJY8IXXW7g5XnO0PHTZAF64LvQIpTN7BglZDMDZ+e1NGyOA/3dJf941mUK4VW42b908nFdNXmZyct+F/Vh4/1jLNllxywwHirXW2wCUUtOBKwHvJeT/ADwN3G+59CgTrUiYJOi420RgqQ7nMjR4rlHoJ8/EcNw4vzt/uPI0Hv50vccx54RPn/5qFGf2asujM9b7nG+F03u0YevBKgD6dM5DKUWzLM8XXrwHQ530am/uirh0UDe2//Ey9h2tdsV3v3XzcOq9Gtbc7ExyszNdoZp/NgSxZOo3Te9PhHA7vJ9oszMzGNi9tYdgO/HW7V7tW/DRHedyWvfWfLjCMX1GltukaLeN6U35sVr6d21Fh7xmBOKXYc4Z8+ld53Hve6v4eKXval5Trh7E69/sYJOfFwXtmOJhdN9OQdNkZqiQxkusiHsPwH2V6BJghHsCpdQQoJfWeqZSKmHEPVrc/u8V8TZBiAEaxYQzuvPwp+s9etMvXj+Efy/ZyRlGT69L62aMcftx3na+Q1zevGm4a9ZOu5hwRjdmrtkXNJ1SyiXsYE08nLx9ywhKfSaYs96UBxsYnDzxNL/T3Z51sqMhu21Mb6Yvb5IcswFId77+3ThbO13eVbh++ElcP/wk11jf7HtG87eFxXy2Zp9PHPywk9vRI8K3S+3Airj7u1Ouy6iUygCeB34eNCOlbgVuBcjpGnw+CiEFSaLHHoXjjcPVj15EKzf/bNc2uR6v3C998ALXtrsQjTk1uKA6e+Y/8HqL1YznrxvMH66MbBGHYOQ1yyLPcOc0y3Q8SeR3aEnFiXpqA4xJDMtvz5ebDwb1t//s3PyAxx+4bAAPXDYgJJvNnmQ+uH0kHYP09N259qyefLxyT0C3FzjmnXnq+4Po0iqXiWd29zj2ocUpEfzx6a9GceSE57xTPdo2D/llRbAm7iWA+ytYPQH3tedaAacDi4wWuyswQyk1UWvttr4aaK1fAV4BaNatb/L8yoW0wf0Ru32eY+DO6Q+OBr3atwjaM5356/NcNmRnZni84h5t2rTIZtrPhzGkVzvqGhsDrlfw0k+GsqPseFizKUaLYCLtzbmndAx4P/5z57nsO+J4qmnTIjvk9wWC4e+N1Ln3jQl5QkSwJu7Lgb5KqQJgD3A98CPnQa31UcAV7KqUWgTc7y3sggDw8KfruSZxfvtJQdC5SaLM9/o3Re8ECvtrkZPl17+eSgw9qR2cFNsyW+RkEcaURsGjZbTW9cBdwBxgI/C+1nq9UupxpdTE0IsUokFdQyMfrIjvPO6CICQOll5i0lrPAmZ57XvEJO3YyM2y52WXWMRdJEZsh2Pel437fBcPiYzAnrNo1D1R5vpRCRO3IwjhkXTTDwiJTWJIc/isLgltCmVBSFRE3IW4EYu3QUNl8n+bYtal7y4kMyLuQtzYX2FvDLgdlFXFZg1eQYg2Iu6Cra4U6esKQmIg4i4IgpCCpLS4J0rkRfJi/9wywUgUP3diWCEI4ZPS4i4IgpCuiLgL7A9zwejUR/rvQvIi4i7w3dbyeJuQmIi2C0mMiLsgmBDOZE2CkCiIuAucSND1VuNNYxJNTywI3qS0uLtHy3hu+0+TzFiph5mXoWiv3XPSBC/TcSzY3DX+j4fiLfFOa/XcVPleCOlLSou7IESCuNyFZCZtxT3R+mXhxndbqYdZ3tGMKQ+Ut9bRl0076pYoMfeCEA5pK+6CEIzK6vp4myAIYSPiLtiK+KoFITEQcRdMCeaWSCa3RTLZKgh2IOIuCH6QpkBIdkTcBUEQUhARd0EwQUYPhGRGxF0QBCEFEXEXBEFIQUTcBUEQUpCEFXc74qVN55ZR9gXG2RXXHW4+yut/oLy9yzDbb9WmQMebjvmmUcr8mGceoZfrncbnv7J2nSVeX0h2ElbcBUEQhPBJG3F376trnXj9skjnlglUH2fe3mWY7Y8E3zx9826aWya8NVqt2Gta5xDmtZEXn4RkJm3EXYgNiddsCkJ6IuIuCIKQgoi4C6bI3DKCkLyIuAuCIKQgKS3u4v8Vwke+O0JyY0nclVKXKKU2KaWKlVKT/By/Tym1QSm1Rik1Xyl1sv2mCkJsEVeOkMwEFXelVCYwFbgUGAjcoJQa6JVsJTBMa30G8CHwtN2GCoIgCNax0nMfDhRrrbdprWuB6cCV7gm01gu11seNj0uAnvaaKQiCIISCFXHvAex2+1xi7DPjZmC2vwNKqVuVUoVKqULrJgqCIAihYkXc/Tke/Y42KaV+AgwDnvF3XGv9itZ6mNZ6mHUTBUEQhFDJspCmBOjl9rknsNc7kVLqAuAh4HytdY095kWG8tj2nDgs1bAy9BfuBGGREXhihEBEMnGYWVrrE4cJQnJjpee+HOirlCpQSuUA1wMz3BMopYYA/wAmaq1L7TfTfhIvEiLcuWWsz7MSzrnhEijvWDStdtQt9boAQjoRVNy11vXAXcAcYCPwvtZ6vVLqcaXURCPZM0Ae8IFSapVSaoZJdkKKI+8WCEJiYMUtg9Z6FjDLa98jbtsX2GyXIAiCEAEp/YaqEBnB+uDR6KNHy1UUylS/gpAKiLgLgiCkICkt7uL/FcJFvjtCspPS4i4IgpCuiLgLggmJFy4rCNaxFC0jCIIQjLq6OkpKSqiuro63KSlBbm4uPXv2JDs7O6zzRdwFQbCFkpISWrVqRX5+PkrJU08kaK0pLy+npKSEgoKCsPIQt4wgCLZQXV1Nhw4dRNhtQClFhw4dInoKSlhxtyNawWM+Ga/9zmORlmNfVEV4+Viph1mapv1m5wUrOzy7lNd/f+eZnWvFLn/lO+eUsXq/JFomPETY7SPSa5mw4i4IgiCET9qIu/bYVq5IiMSJiIhs4rDAE3X5T9O0334C2aW9/gfLw+z8cMoP5X5L3z21yMvLi7cJANxwww2cccYZPP/88xQVFTF48GCGDBnC1q1bbS1HBlQFU2T6ASHdaWhoIDMz07b89u/fz7fffsvOnTsBmDJlCldeeSWTJ0+2rQwnIu6CINjO5P+uZ8PeClvzHNi9NY9ecZrp8aeffprc3Fzuvvtu7r33XlavXs2CBQuYP38+r7/+uitdWVkZV1xxBb///e+5/PLLffJZtGgRkydPplu3bqxatYpZs2YxYcIE1q1bB8Czzz5LVVUVjz32GGPHjmXEiBEsXLiQI0eO8NprrzF69Giqq6u54447KCwsJCsriz//+c+MGzeOiy66iNLSUgYPHsxVV13FSy+9RGZmJl999RULFy609XqJuAuCkBKMGTOG5557jrvvvpvCwkJqamqoq6tj8eLFjB49mk8++YQDBw4wceJEnnjiCS688ELTvJYtW8a6desoKChgx44dAcutr69n2bJlzJo1i8mTJzNv3jymTp0KwNq1aykqKuKiiy5i8+bNzJgxgwkTJrBq1SrAEfKYl5fH/fffb9t1cJLS4i4P4kK4yHcnMgL1sKPFWWedxYoVK6isrKRZs2YMHTqUwsJCvv76a1588UXq6uoYP348U6dO5fzzzw+Y1/Dhwy3Hl1999dWu8p0NweLFi/n1r38NQP/+/Tn55JPZvHkzrVu3Dr+CIZI2A6qCIKQ22dnZ5Ofn8/rrr3PuuecyevRoFi5cyNatWxkwYABZWVmcddZZzJkzJ2heLVu2dG1nZWXR2Njo+uwde96sWTMAMjMzqa+vBxw98ngj4i4IJiROJJVglTFjxvDss88yZswYRo8ezcsvv8zgwYNRSqGUYtq0aRQVFTFlyhTLeXbp0oXS0lLKy8upqalh5syZlux4++23Adi8eTO7du2iX79+YdcrHETcBUFIGUaPHs2+ffsYOXIkXbp0ITc3l9GjR7uOZ2ZmMn36dBYuXMjf//53S3lmZ2fzyCOPMGLECCZMmED//v2DnnPnnXfS0NDAoEGDuO6663jjjTdcPfxYkdI+d0EQ0ovx48dTV1fn+rx582bXdlVVFQA5OTkBXTNjx45l7NixHvvuvvtu7r77bp+0ixYtcm137NjR5XPPzc3ljTfe8Emfn5/viroBeOyxxwLUJjKk5y4IgpCCpHTPPdDcMv62kxkr9YhkrpZwCTznTXjnhmKv2Xw6oZ4npB5r167lpz/9qce+Zs2asXTp0jhZZC8pLe5C7JEhSCFZGDRokCvePBVJW7dMokVChB85Fbwe8eiDBiozFtfenjon1ndEEEIhbcVdCE4wEY6GSEdtbpmo5CoIiYuIuyAIQgoi4i4IgpCCiLgLgiCkICku7uJpFcJDQiGTj4cffpi//OUvrs8PPfQQL774Yhwtii9pHwqZl5sFdcHTCenHgG6xm8Ev5Zg9CfavtTfProPgUvM5YW6++Wauvvpq7rnnHhobG5k+fTrLli2z14YkIsV77sHJynBEZ9w17pQ4WxIeWRnO/2l/K22nY15s5wIRIiM/P58OHTqwcuVKvvjiC4YMGUKHDh3ibVbcsNRzV0pdAvwFyARe1VpP8TreDHgTOAsoB67TWu+wasRLPx7KHW//z2pyW3EG3p1d0B4iXAjluWvP5DcfrI7YJqu0zs3i09vOg39Av655FN18CT9+dSkrdh6mY14zausbqKiu93vuh7ePpGGaed7NsmLXWAw7uR2FOw/HrDzLSJh7+AToYUeTW265hTfeeIP9+/dz0003xcWGRCHoL1gplQlMBS4FBgI3KKUGeiW7GTistT4FeB74k92GmjG+f2db8tFac1r34I/hZ+e3Mz32g7N68tBlA2yxxwpKeapPbnYmPz3nZABGndKBSZea2zIsv73H50cmDOSD20e6Pv/x6kGc0bONjdaac9FpXbh7fF9evGEI5/ZJjJ5W2xY5tM7NjrcZQohcddVVfP755yxfvpyLL7443ubEFSvds+FAsdZ6m9a6FpgOXOmV5krgX8b2h8B45a08fhjXrxOPTBhIZkboXaSTO7QA4LWfn83SB8ez+tGL+O3F/QKKrxMPF4xhpm7aDMhbN48IePzm8wr4vwv6Bs0nr5nnQ1PvTqGvzH7d2b0CHs/KtH5dbzqvgLPz2zO2XycAMjMU2QFcPU98/3TTY/972P/yZdNvPcfv/gyluO/CU5l4Znee/P4g1/7+XVtZMT1k+nYOfq07tMyRjnsSkpOTw7hx4/jhD39o68LWyYgKtmKIUuoa4BKt9S3G558CI7TWd7mlWWekKTE+bzXSlJnle3r35nrVA4PIylDUNjSyo+wY2ZkZtGuZQ2lFNQrom7EHgM2NPTjVbRugb+dWRtme+TZqTXGpY2rPgg7NyT5cDMAhnUd75dhPTiuqaurIU9WuPHu0a0HliTq61u5w5eUsC3CV39ixnyv/Hu2aU1fXQNvj2x2JOvZrMuhgkevcY21OISsjw+HqMPbrjv1RZUWucrrnZZB3fLfZ5fLgSF4fFNC6eTbq+CE4Vuo40Kk/x2rq2XPkBO1a5tAyJ4uSw8ddtrvX6dQurVy2VGfmkdu+JwCHj9dysLKGXu1b0PzIFo9yy5oX0PGEo65H8/rQpmqrj201OotmnU+hoXQTmcrx3dqddRI927VAAQ2NmszyTa70OzNOome75k0NfO0xOBr4OlToFuzX7Vz1KtY9aXT7Hndtk0vryibb9jfLp3l1KW3UccBxD7WG47UN7Dt6wm8Zp2aVwoAr4NrX/R4XfNm4cSMDBsTuydUfjY2NDB06lA8++IC+fYN3shIdf9dUKbVCaz0s2LlWfO7+OjDeLYKVNCilbgVuBejfPY+sLo5J73OAgk6arEyFAioPnyBTKdi7h+o2fchqVgDH6uBYKX0GngXK1yXhJAPo3VHT0KjJzsoAQ9wPdx5B+4PzHYn6jCOnoRE2z6Su5zlkVLekRac8crWmvqiELOohryt9ew1j68Equrdtji7eS0NmLlmd+9OzbQO7Dh2nRZdWDi3fYIh7Z7dJ/N3EvWUPt/Uky4tBZaI694OyImjWmlP7GPdpg5uo5baB6qOO7bwuUHXAdajtSW69Zq1h4wzoeTa07k4LoFWrajq0ziVDQXazYxxQfehy4CsO9xhHns4hJysD8nKgeTvY9R25p37P1Si1BbKq62memwWtOsDuJY5ymrWhY8EZsP0InDhMq16nU7K7BT2r1kKHUxz1AnTfiyEnixLVnZNLF0D+aHq1aHIBZQLH2xTQYtvnUHA+Jzdv63sTneJ+ygVQPM9R/+zmcHgHADUnjSbjRD2NRw6hGmro2XcIpZXVtGmeTU5WBs2zM6G2AIrnsa/rOLq1b832g1W0OTgfep1DRqsuALQCatrVcuR4LY1a09AIbVtkO764bXJhyI99bRMSlg0bNjBhwgSuuuqqlBD2SLEi7iWA+/N/T2CvSZoSpVQW0AY45J2R1voV4BWAYcOGaX74puuYu3fTvbBcoLfbZysPWln4VqyP1+cct3KdThrvvJXbMWe+AC2A4GuxxAcFdHP77L7EbzvjL9j5VpwhGTi+CN7kGv9PDnBuCwv5B6KT8ede5kkmaZ3Xwmyp447Gn5D8DBw4kG3btsXbjITBis99OdBXKVWglMoBrgdmeKWZAfzM2L4GWKATYYVYQRCENCVoz11rXa+UuguYg6NzO01rvV4p9ThQqLWeAbwGvKWUKsbRY78+mkYLgpCYaK1NXaZCaETaP7YU5661ngXM8tr3iNt2NXBtRJYIgpDU5ObmUl5eTocOHUTgI0RrTXl5Obm5ucETm5D20w8IgmAPPXv2pKSkhIMHD8bblJQgNzeXnj39jWxZQ8RdEARbyM7OpqDAbOhaiDUyIYkgCEIKIuIuCIKQgoi4C4IgpCBBpx+IWsFKVQKbgiZMTToCplMzpDhS9/RE6m4fJ2utOwVLFM8B1U1W5kdIRZRShVL39EPqLnWPJeKWEQRBSEFE3AVBEFKQeIr7K3EsO95I3dMTqXt6Epe6x21AVRAEQYge4pYRBEFIQeIi7kqpS5RSm5RSxUqpSfGwwU6UUr2UUguVUhuVUuuVUvcY+9srpeYqpbYY/9sZ+5VS6kWj/muUUkPd8vqZkX6LUupnZmUmGkqpTKXUSqXUTONzgVJqqVGP94zpolFKNTM+FxvH893yeMDYv0kplRQLYCql2iqlPlRKFRn3f2S63Hel1L3G932dUupdpVRuqt53pdQ0pVSpseqcc59t91kpdZZSaq1xzovKjpnXtNYx/cMxbfBWHGtw5ACrgYGxtsPmOnUDhhrbrYDNOBYTfxqYZOyfBPzJ2L4MmI1jbYxzgKXG/vbANuN/O2O7XbzrZ/Ea3Ae8A8w0Pr8PXG9svwzcYWzfCbxsbF8PvGdsDzS+C81wrK2xFciMd70s1PtfwC3Gdg6OxaxS/r4DPYDtQHO3+/3zVL3vwBhgKLDObZ9t9xlYBow0zpkNXBqxzXG4SCOBOW6fHwAeiPfNs7mOnwIX4nhJq5uxrxuO2H6AfwA3uKXfZBy/AfiH236PdIn6h2NRpvnA94CZxhe0DMjyvuc41gUYaWxnGemU9/fAPV2i/gGtDYFTXvtT/r4b4r7bEKos475fnMr3Hcj3Endb7rNxrMhtv0e6cP/i4ZZxfimclBj7UgLjcXMIsBToorXeB2D872wkM7sGyXptXgB+BzQanzsAR7TW9cZn93q46mgcP2qkT8a69wYOAq8bLqlXlVItSYP7rrXeAzwL7AL24biPK0iP++7Ervvcw9j23h8R8RB3S4tpJyNKqTzgI+D/tNYVgZL62acD7E9YlFITgFKt9Qr33X6S6iDHkq7uOHqgQ4GXtNZDgGM4Hs/NSJm6G/7lK3G4UroDLYFL/SRNxfsejFDrGpVrEA9xt7LgdtKhlMrGIexva63/Y+w+oJTqZhzvBpQa+82uQTJem1HARKXUDmA6DtfMC0Bb5VgsHTzr4aqj8lxMPRnrXgKUaK2XGp8/xCH26XDfLwC2a60Paq3rgP8A55Ie992JXfe5BM/15m25BvEQdysLbicVxsj2a8BGrfWf3Q65Lxz+Mxy+eOf+G41R9XOAo8Zj3RzgIqVUO6NndJGxL2HRWj+gte6ptc7HcS8XaK1/DCzEsVg6+Nbd32LqM4DrjaiKAqAvjkGmhEVrvR/YrZTqZ+waD2wgDe47DnfMOUqpFsb331n3lL/vbthyn41jlUqpc4xreaNbXuETp4GJy3BElGwFHor3QIkN9TkPx2PUGmCV8XcZDp/ifGCL8b+9kV4BU436rwWGueV1E1Bs/P0i3nUL8TqMpSlapjeOH2kx8AHQzNifa3wuNo73djv/IeOabMKGaIEY1XkwUGjc+09wREGkxX0HJgNFwDrgLRwRLyl534F3cYwt1OHoad9s530GhhnXcSvwN7wG6cP5kzdUBUEQUhB5Q1UQBCEFEXEXBEFIQUTcBUEQUhARd0EQhBRExF0QBCEFEXEXBEFIQUTcBUEQUhARd0EQhBTk/wNPyKBeyF6HOAAAAABJRU5ErkJggg==\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "# oversampled data \n",
    "data_new = pd.DataFrame({'wk_runoff':X_train_res[:,-1],'y':Y_train_res[:]})\n",
    "data_new.plot()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 81,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.utils import shuffle\n",
    "X_train_res_shuf, Y_train_res_shuf = shuffle( X_train_res, Y_train_res, random_state=0)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 82,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.0    2276\n",
       "1.0      25\n",
       "Name: Flood, dtype: int64"
      ]
     },
     "execution_count": 82,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "Y_test.value_counts()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 83,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Try binary classifiers\n",
    "from pandas.plotting import scatter_matrix\n",
    "\n",
    "from sklearn import model_selection\n",
    "from sklearn.metrics import classification_report\n",
    "from sklearn.metrics import confusion_matrix\n",
    "from sklearn.metrics import accuracy_score\n",
    "from sklearn.linear_model import LogisticRegression\n",
    "from sklearn.tree import DecisionTreeClassifier\n",
    "from sklearn.neighbors import KNeighborsClassifier\n",
    "from sklearn.discriminant_analysis import LinearDiscriminantAnalysis\n",
    "from sklearn.naive_bayes import GaussianNB\n",
    "from sklearn.svm import SVC"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 84,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test options and evaluation metric\n",
    "seed = 7\n",
    "scoring = 'accuracy'"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 85,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "LR: 0.996055 (0.001315)\n",
      "LDA: 0.988258 (0.003007)\n",
      "KNN: 0.985253 (0.002300)\n",
      "CART: 1.000000 (0.000000)\n",
      "NB: 0.993613 (0.001442)\n",
      "SVM: 0.997088 (0.001482)\n"
     ]
    }
   ],
   "source": [
    "# Spot Check Algorithms\n",
    "models = []\n",
    "models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))\n",
    "models.append(('LDA', LinearDiscriminantAnalysis()))\n",
    "models.append(('KNN', KNeighborsClassifier()))\n",
    "models.append(('CART', DecisionTreeClassifier()))\n",
    "models.append(('NB', GaussianNB()))\n",
    "models.append(('SVM', SVC(gamma='auto')))\n",
    "# evaluate each model in turn\n",
    "results = []\n",
    "names = []\n",
    "for name, model in models:\n",
    "\tkfold = model_selection.KFold(n_splits=10, random_state=seed)\n",
    "\tcv_results = model_selection.cross_val_score(model, X_train_res_shuf, Y_train_res_shuf, cv=kfold, scoring=scoring)\n",
    "\tresults.append(cv_results)\n",
    "\tnames.append(name)\n",
    "\tmsg = \"%s: %f (%f)\" % (name, cv_results.mean(), cv_results.std())\n",
    "\tprint(msg)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 96,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAYoAAAEVCAYAAAD+TqKGAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDMuMC4zLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvnQurowAAIABJREFUeJzt3X+YXVV97/H3p8MkKZUfCRkFk5jQkmrSGAKOiIqGYLFBLBGwkhQVfCIUr4ReES/geCWmHSA+CApFc8FESdUBSkVDCw00BkgKtJlIgtCIBKoyBHEgMSH8CBn83j/2mrhznNlz5szMOTPD5/U8+5m914+915ozc75nrb332YoIzMzMuvMHtW6AmZkNbg4UZmZWyIHCzMwKOVCYmVkhBwozMyvkQGFmZoUcKGzASfq2pL8foH2fLunOgvxjJbUNxLGHOkmfl/TNWrfDBj8HCus3ku6WtE3SyGodMyK+GxHvz7UhJB1WreMrc56khyW9IKlN0j9Jemu12lCpiLg0Ij5Z63bY4OdAYf1C0iTgPUAAJ1XpmPtU4zg9+Brwt8B5wBjgT4EfACfWslE9GSS/OxsiHCisv3wceAD4NnBGUUFJ/0fS05K2SPpkfhQg6QBJyyW1S/qFpC9I+oOUd6ak/5B0laStwMKUtjbl35sOsVHSTkmn5Y75WUm/Tsf9RC7925K+LumOVOc/JB0s6atpdPRTSUd004/JwKeBeRHxo4jYFREvplHO5b3sz28kPSHpXSn9ydTeM0raukTSXZKel3SPpIm5/K+lejskrZf0nlzeQkm3SPqOpB3AmSntOyl/VMp7LrVlnaQ3pLw3SlohaaukzZLOKtnvzamPz0t6RFJj0etvQ48DhfWXjwPfTctfdL7JlJI0Gzgf+HPgMGBmSZFrgAOAP055Hwc+kct/B/AE8HqgOV8xIt6bVg+PiNdFxE1p++C0z3HAfOBaSaNzVT8CfAEYC+wC7gd+nLZvAa7sps/vA9oi4r+6yS+3Pw8BBwHfA24E3k72u/ko8A+SXpcrfzrwd6ltG8h+353WATPIRjbfA/5J0qhc/pzUnwNL6kEW3A8AJqS2nAO8lPJagDbgjcCHgUslvS9X96TU7gOBFcA/FPw+bAhyoLA+k3QMMBG4OSLWA48Df91N8Y8A34qIRyLiReBLuf3UAacBF0fE8xHxc+ArwMdy9bdExDUR0RERL1Ge3cCiiNgdEbcDO4E35/JvjYj1EfEycCvwckQsj4hXgZuALkcUZG+oT3d30DL78z8R8a3csSaktu6KiDuBV8iCRqd/jYh7I2IX0AS8U9IEgIj4TkQ8l343XwFGlvTz/oj4QUT8tovf3e7Un8Mi4tX0+9iR9n0McGFEvBwRG4BvlvRhbUTcnvrwj8Dh3f1ObGhyoLD+cAZwZ0Q8m7a/R/fTT28Ensxt59fHAiOAX+TSfkE2EuiqfLmei4iO3PaLQP5T+jO59Ze62M6X3Wu/wCEFxy2nP6XHIiKKjr+n/xGxE9hK9jvtnF7bJGm7pN+QjRDGdlW3C/8IrARuTFOCX5ZUn/a9NSKeL+jDr3LrLwKjfA5keHGgsD6R9Idko4SZkn4l6VfAZ4DDJXX1yfJpYHxue0Ju/VmyT7YTc2lvAp7KbQ+mrzteBYwvmJMvpz+9tef3laakxgBb0vmIC8lei9ERcSCwHVCubre/uzTa+lJETAXeBXyQbJpsCzBG0n792AcbYhworK8+BLwKTCWbH58BTAHWkL3RlLoZ+ISkKZL2Bb7YmZGmLm4GmiXtl07Ung98pxfteYbsfMCAi4jHgK8DLcru1xiRTgrPlXRRP/Wn1AckHSNpBNm5iv+MiCeB/YAOoB3YR9IXgf3L3amkWZLemqbLdpAFuFfTvu8DLkt9m052nqf0HIcNYw4U1ldnkJ1z+GVE/KpzITuheXrpFERE3AFcDawGNpOdOIbsJDLAAuAFshPWa8mmsZb1oj0LgRvSlTsfqbBPvXEeWV+vBX5Ddn7mZOC2lN/X/pT6HnAJ2ZTT28hObkM2bXQH8DOyqaGX6d003cFkJ7p3AJuAe/hdQJsHTCIbXdwKXBIRd/WhDzbEyA8uslqSNAV4GBhZch7BSkj6NtlVVl+odVvstcUjCqs6SSenaZrRwGLgNgcJs8HLgcJq4W/I5tIfJzu/8anaNsfMinjqyczMCnlEYWZmhRwozMyskAOFmZkVcqAwM7NCDhRmZlbIgcLMzAo5UJiZWSEHCjMzK+RAYWZmhRwozMyskAOFmZkVcqAwM7NCDhRmZlbIgcLMzArt03ORwW/s2LExadKkWjfDzGxIWb9+/bMR0dBTuWERKCZNmkRra2utm2FmNqRI+kU55Tz1ZGZmhRwozMyskAOFmZkVcqAwM7NCDhRmZlaorEAhaZmkX0t6uJt8Sbpa0mZJD0k6Mpd3hqTH0nJGLv1tkn6S6lwtSSl9jKS7Uvm7JI3uayfNzKxy5Y4ovg3MLsg/AZiclrOBb0D2pg9cArwDOAq4JPfG/41UtrNe5/4vAlZFxGRgVdo2M7MaKStQRMS9wNaCInOA5ZF5ADhQ0iHAXwB3RcTWiNgG3AXMTnn7R8T9ERHAcuBDuX3dkNZvyKWbmVkN9NcNd+OAJ3PbbSmtKL2ti3SAN0TE0wAR8bSk13d1QElnk41IeNOb3tQPXTAbZBYeUOXjba/u8WzI6K9AoS7SooL0skXEdcB1AI2Njb2qazYk+I3bBon+uuqpDZiQ2x4PbOkhfXwX6QDPpKkp0s9f91MbzcysAv0VKFYAH09XPx0NbE/TRyuB90sanU5ivx9YmfKel3R0utrp48APc/vqvDrqjFy6mZnVQFlTT5JagGOBsZLayK5kqgeIiCXA7cAHgM3Ai8AnUt5WSX8HrEu7WhQRnSfFP0V2NdUfAnekBeBy4GZJ84FfAn9VeffMzKyvlF10NLQ1NjaGvz3WzKx3JK2PiMaeyvnObDMzK+RAYWZmhRwozMyskAOFmZkVGhaPQjUzGyrS959WpFYXHzlQmJlVUdGbvaSaBYMinnoyM7NCDhRmZlbIgcLMzAo5UJiZWSEHCjMzK+RAYWZmhRwozMyskAOFmZkV8g133RiKd0+a2eAwZswYtm3bVlHdSt57Ro8ezdatW3suWCEHim4MxbsnzWxw2LZtW1XfI/rywbYcnnoyM7NCZQUKSbMlPSpps6SLusifKGmVpIck3S1pfC5vsaSH03JaLn2NpA1p2SLpByn9WEnbc3lf7I+OmplZZXqcepJUB1wLHA+0AeskrYiI/84VuwJYHhE3SDoOuAz4mKQTgSOBGcBI4B5Jd0TEjoh4T+4Y/wz8MLe/NRHxwb52zszM+q6cEcVRwOaIeCIiXgFuBOaUlJkKrErrq3P5U4F7IqIjIl4ANgKz8xUl7QccB/ygsi6YmdlAKidQjAOezG23pbS8jcCpaf1kYD9JB6X0EyTtK2ksMAuYUFL3ZGBVROzIpb1T0kZJd0j6szL7YmZmA6CcQNHV6fTS0/kXADMlPQjMBJ4COiLiTuB24D6gBbgf6CipOy/ldfoxMDEiDgeuoZuRhqSzJbVKam1vby+jG2ZmVolyAkUbe48CxgNb8gUiYktEnBIRRwBNKW17+tkcETMi4niyoPNYZ7006jgK+NfcvnZExM60fjtQn0Yje4mI6yKiMSIaGxoayuutmZn1WjmBYh0wWdKhkkYAc4EV+QKSxkrq3NfFwLKUXpeCAZKmA9OBO3NV/wr4l4h4Obevg5UuCpZ0VGrjc5V0zszM+q7Hq54iokPSucBKoA5YFhGPSFoEtEbECuBY4DJJAdwLfDpVrwfWpPf9HcBHIyI/9TQXuLzkkB8GPiWpA3gJmBsDdOfKcLt70swGh7hkf1h4QHWPN4A0HO4wbmxsjNbW1l7Xq/Yd1r6j2+y1Yai8t0haHxGNPZXzndlmZlbIgcLMzAo5UJiZWSEHCjMzK+RAYWZmhfw8CjOzATDQz4jIGz169IDu34HCzKyfVXpp7GC9hN5TT2ZmVug1PaIYbndPmg0Hw/159T31ryi/Vv17TQcKfWlH9e+eXFi1w5kNScP9efVDsf2eejIzs0IOFGZmVsiBwszMCjlQmJlZIQcKMzMr5EBhZmaFHCjMzKyQA4WZmRUqK1BImi3pUUmbJV3URf5ESaskPSTpbknjc3mLJT2cltNy6d+W9D+SNqRlRkqXpKvTsR6SdGR/dNTMzCrTY6CQVAdcC5wATAXmSZpaUuwKYHlETAcWAZeluicCRwIzgHcAn5OU/x6Lz0XEjLRsSGknAJPTcjbwjUo7Z2ZmfVfOiOIoYHNEPBERrwA3AnNKykwFVqX11bn8qcA9EdERES8AG4HZPRxvDlnQiYh4ADhQ0iFltNPMzAZAOYFiHPBkbrstpeVtBE5N6ycD+0k6KKWfIGlfSWOBWcCEXL3mNL10laSRvTgeks6W1Cqptb29vYxumJlZJcoJFF19lWHpt1pdAMyU9CAwE3gK6IiIO4HbgfuAFuB+oCPVuRh4C/B2YAxwYS+OR0RcFxGNEdHY0NBQRjfMzKwS5QSKNvYeBYwHtuQLRMSWiDglIo4AmlLa9vSzOZ2DOJ4sCDyW0p9O00u7gG+RTXGVdTwzM6uecgLFOmCypEMljQDmAivyBSSNldS5r4uBZSm9Lk1BIWk6MB24M20fkn4K+BDwcKq/Avh4uvrpaGB7RDzdhz6amVkf9Pg8iojokHQusBKoA5ZFxCOSFgGtEbECOBa4TFIA9wKfTtXrgTXpQRw7gI9GROfU03clNZCNMjYA56T024EPAJuBF4FP9LmXBYbTc23NzAaChuJDNEo1NjZGa2tr1Y43HB6eYjYU+X+vf0laHxGNPZXzndlmZlboNf0o1CJD8bm2ZkPFmDFj2LZtW0V1K5kuHj16NFu3bq3oeOZA0S2/2ZsNnG3btlX9efVWOU89mZlZIQcKMzMr5EBhZmaFHCjMzKyQA4WZWY21tLQwbdo06urqmDZtGi0tLbVu0l581ZOZWQ21tLTQ1NTE0qVLOeaYY1i7di3z588HYN68eTVuXcYjCjOzGmpubmbp0qXMmjWL+vp6Zs2axdKlS2lubq510/bwV3iYWfUtPKAGx9xe/WOWoa6ujpdffpn6+vo9abt372bUqFG8+uqrA3rscr/Cw1NPZlZ1+tKOqt9wFwurdrhemTJlCmvXrmXWrFl70tauXcuUKVNq2Kq9eerJzKyGmpqamD9/PqtXr2b37t2sXr2a+fPn09TUVOum7eERhZlZDXWesF6wYAGbNm1iypQpNDc3D5oT2eBzFGZWA9X+unB/PXnX/DXjZmbWLxwozMysUFmBQtJsSY9K2izpoi7yJ0paJekhSXdLGp/LWyzp4bSclkv/btrnw5KWSapP6cdK2i5pQ1q+2B8dtZ4tWLCAUaNGIYlRo0axYMGCWjfJzAaBHgOFpDrgWuAEYCowT9LUkmJXAMsjYjqwCLgs1T0ROBKYAbwD+Jyk/VOd7wJvAd4K/CHwydz+1kTEjLQsqrRzVr4FCxawZMkSLr30Ul544QUuvfRSlixZ4mBhA0ZS1RY/r75vyhlRHAVsjognIuIV4EZgTkmZqcCqtL46lz8VuCciOiLiBWAjMBsgIm6PBPgvYDxWM9dffz2LFy/m/PPPZ9999+X8889n8eLFXH/99bVumg1DEVHRUmldP92ub8oJFOOAJ3PbbSktbyNwalo/GdhP0kEp/QRJ+0oaC8wCJuQrpimnjwH/lkt+p6SNku6Q9GddNUrS2ZJaJbW2t7eX0Q0rsmvXLs4555y90s455xx27dpVoxaZ2WBRTqDo6hmCpdeZXQDMlPQgMBN4CuiIiDuB24H7gBbgfqCjpO7XgXsjYk3a/jEwMSIOB64BftBVoyLiuohojIjGhoaGMrphRUaOHMmSJUv2SluyZAkjR46sUYvMbLAoJ1C0sfcoYDywJV8gIrZExCkRcQTQlNK2p5/N6VzD8WRB57HOepIuARqA83P72hERO9P67UB9Go3YADrrrLO48MILufLKK3nxxRe58sorufDCCznrrLNq3TQzq7Fy7sxeB0yWdCjZSGEu8Nf5AumNfGtE/Ba4GFiW0uuAAyPiOUnTgenAnSnvk8BfAO9L9Tr3dTDwTESEpKPIgtlzfeum9eSaa64B4POf/zyf/exnGTlyJOecc86edDN77SrrzmxJHwC+CtQByyKiWdIioDUiVkj6MNmVTgHcC3w6InZJGkU2lQSwAzgnIjakfXYAvwCeT/nfj4hFks4FPkU2RfUScH5E3FfUPt+Zbfba4Dus+1e5d2b7KzzMbMhwoOhf/goPMzPrFw4UZmZWyIHCzMwKOVCYmVkhP7jIzAYVqat7fMvL94nugeFAYWaDit/sBx9PPZmZWSEHCjMzK+RAYWZmhRwozMyskAOFmZkVcqAwM7NCDhRmZlbIgcLMzAo5UJiZWSEHCjMzK+RAYWZmhcoKFJJmS3pU0mZJF3WRP1HSKkkPSbpb0vhc3mJJD6fltFz6oZL+U9Jjkm6SNCKlj0zbm1P+pL5308zMKtVjoJBUB1wLnABMBeZJmlpS7ApgeURMBxaRPT8bSScCRwIzgHcAn5O0f6qzGLgqIiYD24D5KX0+sC0iDgOuSuXMzKxGyhlRHAVsjognIuIV4EZgTkmZqcCqtL46lz8VuCciOiLiBWAjMFvZ9wQfB9ySyt0AfCitz0nbpPz3qafvHTYzswFTTqAYBzyZ225LaXkbgVPT+snAfpIOSuknSNpX0lhgFjABOAj4TUR0dLHPPcdL+dtTeTMzq4FyAkVXn+ZLvzD+AmCmpAeBmcBTQEdE3AncDtwHtAD3Ax097LOc4yHpbEmtklrb29vL6IaZmVWinEDRRjYK6DQe2JIvEBFbIuKUiDgCaEpp29PP5oiYERHHkwWBx4BngQMl7dPFPvccL+UfAGwtbVREXBcRjRHR2NDQUFZnzcys98oJFOuAyekqpRHAXGBFvoCksZI693UxsCyl16UpKCRNB6YDd0b2CKvVwIdTnTOAH6b1FWmblP+j8COvzMxqpsdHoUZEh6RzgZVAHbAsIh6RtAhojYgVwLHAZZICuBf4dKpeD6xJ56J3AB/NnZe4ELhR0t8DDwJLU/pS4B8lbSYbScztezfttaYv1z/4c4nZ3jQc/ikaGxujtbW11s2wIUKSg4EZIGl9RDT2VM53ZpuZWSEHCjMzK+RAYWZmhRwozMyskAOFmZkVcqAwM7NCDhRmZlbIgcLMzAr1eGe2DWELD6jBMbdX/5hmNqAcKIYxfWlHVe9AlkQsrNrhzKxKPPVkZmaFHCjMzKyQA4WZmRVyoDAzs0IOFGZmVsiBwszMCjlQmJlZobIChaTZkh6VtFnSRV3kT5S0StJDku6WND6X92VJj0jaJOlqZfaTtCG3PCvpq6n8mZLac3mf7L/umplZb/V4w52kOuBa4HigDVgnaUVE/Heu2BXA8oi4QdJxwGXAxyS9C3g3MD2VWwvMjIi7gRm5Y6wHvp/b300RcW7l3TIzs/5SzojiKGBzRDwREa8ANwJzSspMBVal9dW5/ABGASOAkUA98Ey+oqTJwOuBNZV0wMzMBlY5gWIc8GRuuy2l5W0ETk3rJwP7STooIu4nCxxPp2VlRGwqqTuPbASR/66JU9M01i2SJpTZFzMzGwDlBAp1kVb6BUIXADMlPQjMBJ4COiQdBkwBxpMFl+Mkvbek7lygJbd9GzApIqYD/w7c0GWjpLMltUpqbW9vL6MbZmZWiXICRRuQ/1Q/HtiSLxARWyLilIg4AmhKadvJRhcPRMTOiNgJ3AEc3VlP0uHAPhGxPrev5yJiV9q8HnhbV42KiOsiojEiGhsaGsrohpmZVaKcQLEOmCzpUEkjyEYAK/IFJI2V1Lmvi4Flaf2XZCONfSTVk4028lNP89h7NIGkQ3KbJ5WUNzOzKuvxqqeI6JB0LrASqAOWRcQjkhYBrRGxAjgWuExSAPcCn07VbwGOA35CNl31bxFxW273HwE+UHLI8ySdBHQAW4EzK+ybmZn1A1XzeQUDpbGxMVpbW2vdjEFH6ur00sAZPXo0W7dureoxKyGpqs/pMBusJK2PiMaeyvnBRcNYpW+GfiM1szx/hYeZmRXyiOI1qqdpqaJ8jzbMXlscKF6j/GZvZuXy1JMNWWPGjEFSrxegonpjxoypcY/NasMjChuytm3bVtWRUbWvIjMbLDyiMDOzQg4UZmZWyIHCzMwKOVCYmVkhBwozMyvkQGFmZoUcKMzMrJADhZmZFXKgMDOzQg4UZmZWyIHCzMwK+buebMiKS/aHhQdU93hmr0FlBQpJs4GvkT0z+5sRcXlJ/kRgGdBA9pzrj0ZEW8r7MnAi2ejlLuBvIyIk3Q0cAryUdvP+iPi1pJHAcuBtwHPAaRHx87500oYnfWlH1b8UMBZW7XBmg0aPU0+S6oBrgROAqcA8SVNLil0BLI+I6cAi4LJU913Au4HpwDTg7cDMXL3TI2JGWn6d0uYD2yLiMOAqYHGlnTMzs74r5xzFUcDmiHgiIl4BbgTmlJSZCqxK66tz+QGMAkYAI4F64JkejjcHuCGt3wK8T/5+ZzOzmiknUIwDnsxtt6W0vI3AqWn9ZGA/SQdFxP1kgePptKyMiE25et+StEHS/80Fgz3Hi4gOYDtwUGmjJJ0tqVVSa3t7exndMDOzSpQTKLr6NF86MXwBMFPSg2RTS08BHZIOA6YA48kCwHGS3pvqnB4RbwXek5aP9eJ4RMR1EdEYEY0NDQ1ldMPMzCpRTqBoAybktscDW/IFImJLRJwSEUcATSltO9no4oGI2BkRO4E7gKNT/lPp5/PA98imuPY6nqR9gAPITpCbmVkNlBMo1gGTJR0qaQQwF1iRLyBprKTOfV1MdgUUwC/JRhr7SKonG21sSttjU9164IPAw6nOCuCMtP5h4EdRzUtbzMxsLz0GinSe4FxgJbAJuDkiHpG0SNJJqdixwKOSfga8AWhO6bcAjwM/ITuPsTEibiM7sb1S0kPABrKpqutTnaXAQZI2A+cDF/W5l2ZmVjENhw/rjY2N0draWutmWJVJqv59FMPg/8Wsk6T1EdHYUzl/hYeZmRVyoDAzs0L+ricb0qp5L+bo0aOrdiyzwcSBwoasSs8X+FyDWe946snMzAo5UJiZWSEHCjMzK+RAYWZmhRwozMyskAOFmZkVcqAwM7NCDhRmZlbIgcLMzAo5UJiZWSEHCjMzK+RAYWZmhRwozMysUFmBQtJsSY9K2izp9x5NKmmipFWSHpJ0t6TxubwvS3pE0iZJVyuzr6R/lfTTlHd5rvyZktolbUjLJ/unq2ZmVokeA4WkOuBa4ARgKjBP0tSSYlcAyyNiOrAIuCzVfRfwbmA6MA14OzCzs05EvAU4Ani3pBNy+7spImak5ZsV987MzPqsnBHFUcDmiHgiIl4BbgTmlJSZCqxK66tz+QGMAkYAI4F64JmIeDEiVgOkff4YGI+ZmQ065QSKccCTue22lJa3ETg1rZ8M7CfpoIi4nyxwPJ2WlRGxKV9R0oHAX/K7QANwaprGukXShK4aJelsSa2SWtvb28vohpmZVaKcQNHVsyZLHw92ATBT0oNkU0tPAR2SDgOmkI0WxgHHSXrvnh1L+wAtwNUR8URKvg2YlKax/h24oatGRcR1EdEYEY0NDQ1ldMPMzCpRTqBoA/Kf6scDW/IFImJLRJwSEUcATSltO9no4oGI2BkRO4E7gKNzVa8DHouIr+b29VxE7Eqb1wNv62WfzMysH5UTKNYBkyUdKmkEMBdYkS8gaaykzn1dDCxL678kG2nsI6mebLSxKdX5e+AA4H+X7OuQ3OZJneXNzKw2egwUEdEBnAusJHvTvjkiHpG0SNJJqdixwKOSfga8AWhO6bcAjwM/ITuPsTEibkuXzzaRnQT/ccllsOelS2Y3AucBZ/ZDP83MrEKKKD3dMPQ0NjZGa2trrZthQ4QkhsPfvVlfSVofEY09lfOd2WZmVsiBwszMCjlQmJlZIQcKMzMr5EBhZmaFHChsj5aWFqZNm0ZdXR3Tpk2jpaWl1k0ys0Fgn1o3wAaHlpYWmpqaWLp0Kccccwxr165l/vz5AMybN6/GrTOzWvKIwgBobm5m6dKlzJo1i/r6embNmsXSpUtpbm7uubKZDWu+4c4AqKur4+WXX6a+vn5P2u7duxk1ahSvvvpqDVtWGamr77Isz3D4nzArh2+4s16ZMmUKa9eu3Stt7dq1TJkypUYt6puIqHgxs705UBgATU1NzJ8/n9WrV7N7925Wr17N/PnzaWpqqnXTzKzGfDLbgN+dsF6wYAGbNm1iypQpNDc3+0S2mfkchZnZa5XPUZiZWb9woDAzs0IOFGZmVsiBwszMCjlQmJlZoWFx1ZOkduAXVTzkWODZKh6v2ty/oW0492849w2q37+JEdHQU6FhESiqTVJrOZeUDVXu39A2nPs3nPsGg7d/nnoyM7NCDhRmZlbIgaIy19W6AQPM/RvahnP/hnPfYJD2z+cozMyskEcUZmZWyIGiB5J2dpG2UNJTkjZI+m9JQ+YrVsvoz2OSvi9pakmZBkm7Jf1N9VrbO/m+SfpA6subUv9elPT6bsqGpK/kti+QtLBqDe+BpIMl3Sjp8fT3drukP015n5H0sqQDcuWPlbRd0oOSfirpipT+ifQab5D0iqSfpPXLa9W37hS9JiV/rz+V9A1Jg/69TFKTpEckPZTafoeky0rKzJC0Ka3/XNKakvwNkh6uZrvBgaIvroqIGcAc4P9Jqu+pwiB3VUTMiIjJwE3AjyTlr6/+K+ABYNAHRUnvA64BZkfEL1Pys8Bnu6myCzhF0thqtK83lD2q71bg7oj4k4iYCnweeEMqMg9YB5xcUnVNRBwBHAF8UNK7I+Jb6TWeAWwBZqXti6rTm17p6TXp/P+bCrwVmFm1llVA0juBDwJHRsR04M+By4HTSorOBb6X295P0oS0j5o9RcyBoo8i4jHgRWB0rdvSXyLiJuBO4K9zyfPI3mjHSxpXk4aVQdJ7gOuBEyPi8VzWMuA0SWO6qNZBdhLxM1VoYm/NAnZHxJLOhIjYEBFrJP0J8DrgC3QTwCPgNi6cAAADR0lEQVTiJWADMGhfs26U+5qMAEYB2wa8RX1zCPBsROwCiIhnI+Ie4DeS3pEr9xHgxtz2zfwumMwDWqrR2FIOFH0k6UjgsYj4da3b0s9+DLwFIH2iOTgi/ou9/3AHm5HAD4EPRcRPS/J2kgWLv+2m7rXA6fkpnEFiGrC+m7zON441wJvzU2udJI0GJgP3DlgLB07Ra/IZSRuAp4GfRcSG6jat1+4EJkj6maSvS+ocAbWQjSKQdDTwXPrw2ekW4JS0/pfAbdVqcJ4DReU+I+lR4D+BhTVuy0BQbn0uWYCA7NPOYJ1+2g3cB8zvJv9q4AxJ+5dmRMQOYDlw3sA1r9/NBW6MiN8C3yebHuz0HkkPAb8C/iUiflWLBvZFD69J59TT64E/kjS3qo3rpYjYCbwNOBtoB26SdCbZ/9OH0zmWufz+iGErsC31bxPZ7EXVOVBU7qqIeDPZp+vlkkbVukH97AiyP0zIAsOZkn4OrAAOlzS5Vg0r8FuyofvbJX2+NDMifkM2//u/uqn/VbIg80cD1sLee4TsDWYvkqaTjRTuSq/LXPYO4GvSXPhbgU9JmlGFtg6EwtckInYD/wa8t5qNqkREvBoRd0fEJcC5wKkR8STwc7JzLKfyuw9keTeRja5qMu0EDhR9FhHfB1qBM2rdlv4i6VTg/UCLpDcDfxQR4yJiUkRMAi4jDZcHm4h4keyk4emSuhpZXAn8DV08Lz4itpL9o3Y3IqmFHwEjJZ3VmSDp7cDXgIWdr0lEvBEYJ2livnJE/Izs9bqwmo3uLz29Julk/7uAx7vKHywkvbnkw9UMfvdFpi3AVcDjEdHWRfVbgS8DKwe2ld1zoOjZvpLacsv5XZRZBJw/FC7Ro/v+fKbz8ljgo8BxEdFO9in11pJ9/DODd/qp881lNvAFSXNK8p4l68/Ibqp/hewbPAeFyO6IPRk4Pl0e+wjZVOex/P7rcitdB/AlwHslHTqATR1IXb0mnecoHiYL+l+veqt653XADeny5ofIrtZamPL+Cfgz9j6JvUdEPB8RiyPilaq0tAu+M9vMzAoNhU/AZmZWQw4UZmZWyIHCzMwKOVCYmVkhBwozMyvkQGFmZoUcKMzMrJADhZmZFfr/qOtFPh7qnIUAAAAASUVORK5CYII=\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "# Compare Algorithms\n",
    "fig = plt.figure()\n",
    "fig.suptitle('Algorithm Comparison')\n",
    "ax = fig.add_subplot(111)\n",
    "plt.boxplot(results)\n",
    "ax.set_xticklabels(names)\n",
    "plt.savefig('static/img/modelComparison/flood_models.png')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Make predictions on validation dataset"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 97,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "0.970013037809648\n",
      "[[2211   65]\n",
      " [   4   21]]\n",
      "              precision    recall  f1-score   support\n",
      "\n",
      "         0.0       1.00      0.97      0.98      2276\n",
      "         1.0       0.24      0.84      0.38        25\n",
      "\n",
      "   micro avg       0.97      0.97      0.97      2301\n",
      "   macro avg       0.62      0.91      0.68      2301\n",
      "weighted avg       0.99      0.97      0.98      2301\n",
      "\n"
     ]
    }
   ],
   "source": [
    "# Make predictions on validation dataset\n",
    "knn = KNeighborsClassifier()\n",
    "knn.fit(X_train_res_shuf, Y_train_res_shuf)\n",
    "predictions = knn.predict(X_test)\n",
    "print(accuracy_score(Y_test, predictions))\n",
    "cnf_matrix_tra = confusion_matrix(Y_test, predictions)\n",
    "print(cnf_matrix_tra )\n",
    "print(classification_report(Y_test, predictions))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 98,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "0.979574098218166\n",
      "[[2229   47]\n",
      " [   0   25]]\n",
      "              precision    recall  f1-score   support\n",
      "\n",
      "         0.0       1.00      0.98      0.99      2276\n",
      "         1.0       0.35      1.00      0.52        25\n",
      "\n",
      "   micro avg       0.98      0.98      0.98      2301\n",
      "   macro avg       0.67      0.99      0.75      2301\n",
      "weighted avg       0.99      0.98      0.98      2301\n",
      "\n"
     ]
    }
   ],
   "source": [
    "#-----------------------LinearDiscriminantAnalysis---------------------------------\n",
    "from sklearn.discriminant_analysis import LinearDiscriminantAnalysis\n",
    "clf1=LinearDiscriminantAnalysis()\n",
    "clf1.fit(X_train_res_shuf, Y_train_res_shuf)\n",
    "predictions = clf1.predict(X_test)\n",
    "# y_predict3=clf1.predict(x_test)\n",
    "print(accuracy_score(Y_test, predictions))\n",
    "cnf_matrix_tra = confusion_matrix(Y_test, predictions)\n",
    "print(cnf_matrix_tra )\n",
    "print(classification_report(Y_test, predictions))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 99,
   "metadata": {},
   "outputs": [],
   "source": [
    "import itertools\n",
    "\n",
    "def plot_confusion_matrix(cm, classes,\n",
    "                          normalize=False,\n",
    "                          title='Confusion matrix',\n",
    "                          cmap=plt.cm.Blues):\n",
    "    \"\"\"\n",
    "    This function prints and plots the confusion matrix.\n",
    "    Normalization can be applied by setting `normalize=True`.\n",
    "    \"\"\"\n",
    "    plt.imshow(cm, interpolation='nearest', cmap=cmap)\n",
    "    plt.title(title)\n",
    "    plt.colorbar()\n",
    "    tick_marks = np.arange(len(classes))\n",
    "    plt.xticks(tick_marks, classes, rotation=0)\n",
    "    plt.yticks(tick_marks, classes)\n",
    "\n",
    "    if normalize:\n",
    "        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]\n",
    "        #print(\"Normalized confusion matrix\")\n",
    "    else:\n",
    "        1#print('Confusion matrix, without normalization')\n",
    "\n",
    "    #print(cm)\n",
    "\n",
    "    thresh = cm.max() / 2.\n",
    "    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):\n",
    "        plt.text(j, i, cm[i, j],\n",
    "                 horizontalalignment=\"center\",\n",
    "                 color=\"white\" if cm[i, j] > thresh else \"black\")\n",
    "\n",
    "    plt.tight_layout()\n",
    "    plt.ylabel('True label')\n",
    "    plt.xlabel('Predicted label')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 101,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Recall metric in the train dataset: 100.0%\n"
     ]
    },
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAVMAAAEmCAYAAADfpHMGAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDMuMC4zLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvnQurowAAIABJREFUeJzt3XucX9O9//HXOwlBUZeQRkLjEnHJqZCI/LRVRSNUhT5oo447KYc6jh6Xas+hlGqrehqUUo66JaKo0CipHlUlJNIgqSAhZCREXBP3xOf3x16TfhMz3/l+Z/Z39sx33s889mO+e+2111p7JvOZtde+LEUEZmbWNt2KboCZWT1wMDUzy4GDqZlZDhxMzcxy4GBqZpYDB1Mzsxw4mHZQktaUdKektyTd0oZyDpV0b55tK4qkL0p6ugbl5vK9tq7NwbSNJH1L0jRJSyUtlHS3pC/kUPRBQG9gw4g4uLWFRMSNETEih/bUlKSQtFW5PBHx14gYWIPqy36vJZ0j6YamdpQ0T9J7kpZIelPSQ5KOl/SJ361UTkgalv8hWNEcTNtA0qnA/wAXkP0ybgb8ChiVQ/GfBZ6JiGU5lNXpSepRw+Lb+r3+WkSsk8q5EDgDuLo0gyQBhwGvA0e0oa3WUUWEl1YswKeBpcDBZfL0JAu2C9LyP0DPtG13oAH4LrAIWAgclbb9EPgQ+CjVcQxwDnBDSdn9gQB6pPUjgeeAJcDzwKEl6Q+W7LcrMBV4K33dtWTb/cB5wN9SOfcCvZo5tsb2n17S/gOAfYFnyILGWSX5hwEPA2+mvJcCq6dtD6RjeScd7zdLyj8DeBm4vjEt7bNlqmOntL4JsBjYvZn2bpuO701gFrB/c9/rJvZd6Xu/yrZ5wF6rpA0DPgYGlaTtBrwH/CvwWuOxe6mfpfAGdNYFGAksawxmzeQ5F5gCbAxsBDwEnJe27Z72PxdYLQWhd4H10/ZVg+eq6/1TAOoBfAp4GxiYtvUBtk+fjyQFU2AD4A2yHlIP4JC0vmHafj8wF9gaWDOtX9jMsTW2/79T+48DXgVuAtYBtgfeB7ZI+YcAw1O9/YGngFNKygtgqybK/wnZH6U1KQmmKc9xqZy1gHuAi5pp62rAHOAsYHVgD7I/FgOb+t42sX+z22kimKb0F4ETStavBiaktrwGfL3o/8Ne8l18mt96GwKLo/yp4aHAuRGxKCJeJesFHVay/aO0/aOImETWM2rtmODHwCBJa0bEwoiY1USerwLPRsT1EbEsIsYBs4GvleT534h4JiLeI/vlH1ymzo+A8yPiI2A80Av4ZUQsSfXPAj4HEBGPRcSUVO884NfAlyo4prMj4oPUnpVExFXAs8AjZH9Avt9MOcOBtcn+MHwYEX8G7iL7Y1IrC8j+eCFpLeBg4Kb0vfodPtWvOw6mrfca0KuFsbxNgBdK1l9IaSvKWCUYv0v2S1+ViHiH7NT4eGChpD9I2qaC9jS2qW/J+stVtOe1iFiePjcGu1dKtr/XuL+krSXdJellSW+TjTP3KlM2wKsR8X4Lea4CBgGXRMQHzeTZBJgfER+XpK163HnrSzYMAXAgWS97Ulq/EdhH0kY1rN/amYNp6z1Mdhp7QJk8C8guSjTaLKW1xjtkp7ONPlO6MSLuiYivkPXQZpMFmZba09iml1rZpmpcTtauARGxLtkpt1rYp+wrzSStTTYOfTVwjqQNmsm6ANh0lSvsNTtuSTuTBdMHU9IRZH9UXpT0MnAL2el+LXvG1s4cTFspIt4iGy+8TNIBktaStJqkfST9NGUbB/xA0kaSeqX8Td5iU4EZwG6SNpP0aeB7jRsk9Za0v6RPAR+QDRcsb6KMScDW6XauHpK+CWxHdspba+uQjesuTb3mE1bZ/gqwRZVl/hJ4LCKOBf4AXNFMvkfI/hidnn5Gu5MNbYyvoq5uktYoWXqumkHSupL2S+XeEBFPSuoL7AnsRzZkMhjYgWws2Kf6dcTBtA0i4mLgVOAHZBdf5gMnAb9PWX4ETAOeAJ4Epqe01tQ1Gbg5lfUYKwfAbmR3BSwgO7X8EvBvTZTxGtkv9XfJhilOB/aLiMWtaVOV/hP4FtmFn6vIjqXUOcBv072a32ipMEmjyC4CHp+STgV2knToqnkj4kNgf2Afsiv+vwIOj4jZVbT/ELJhi8Zlbsm2OyUtIfv5fx+4GDgqbTsMmBER90bEy40LMBb4nKRBVbTBOjBF+OXQZmZt5Z6pmVkOHEzNzHLgYGpmlgMHUzOzHNTy5RFVU481Q6uvU3QzLEeDt92s6CZYjl58YR6LFy9u6f7ginVf97MRyz7xcFuz4r1X74mIkXnVn6eOFUxXX4eeA1u8K8Y6kQceGlt0EyxHu+2a79sDY9l7Vf3Ovz/jspaemitMhwqmZtbVCD756tdOycHUzIojQLmNGhTKwdTMiuWeqZlZWwm6dS+6EblwMDWzYvk038ysjYRP883M2k7umZqZ5cI9UzOzHLhnambWVr5p38ys7XzTvplZTtwzNTNrK5/mm5m1nYDufgLKzKztPGZqZtZWPs03M8uHe6ZmZjlwz9TMrI1UP8/m18efBDPrvNSt8qWloqRNJf2fpKckzZL07yl9A0mTJT2bvq6f0iVprKQ5kp6QtFNJWUek/M9KOqKluh1MzaxYjb3TSpaWLQO+GxHbAsOBEyVtB5wJ3BcRA4D70jrAPsCAtIwBLs+apA2As4FdgGHA2Y0BuDkOpmZWIOXaM42IhRExPX1eAjwF9AVGAb9N2X4LHJA+jwKui8wUYD1JfYC9gckR8XpEvAFMBspOMe0xUzMrVnVjpr0kTStZvzIirmy6WPUHdgQeAXpHxELIAq6kjVO2vsD8kt0aUlpz6c1yMDWz4lT/pv3FETG0xWKltYFbgVMi4m01H7Cb2hBl0pvl03wzK1CaUK/SpZISpdXIAumNEXFbSn4lnb6Tvi5K6Q3ApiW79wMWlElvloOpmRUr36v5Aq4GnoqIi0s2TQQar8gfAdxRkn54uqo/HHgrDQfcA4yQtH668DQipTXLp/lmVqx87zP9PHAY8KSkGSntLOBCYIKkY4AXgYPTtknAvsAc4F3gKICIeF3SecDUlO/ciHi9XMUOpmZWHOX7bH5EPEjT450AezaRP4ATmynrGuCaSut2MDWzYtXJE1AOpmZWqDJX2jsVB1MzK0w2BZSDqZlZ24jmRzg7GQdTMyuQ3DM1M8uDg6mZWQ66dauPZ4ccTM2sOB4zNTNrO3nM1MwsHw6mZmY5cDA1M8uBg6mZWVv5ApSZWT7cMzUzayNfzTczy4mDqZlZWwnUrT6CaX08x2VmnZakipcKyrpG0iJJM0vSbpY0Iy3zGqczkdRf0nsl264o2WeIpCclzZE0VhVU7p6pmRUq59P8a4FLgesaEyLimyV1/Rx4qyT/3IgY3EQ5lwNjgClk80SNBO4uV7F7pmZWmMYLUHn1TCPiAaDJie9S7/IbwLiybcqmgl43Ih5Oc0RdBxzQUt0OpmZWLFWxtM0XgVci4tmStM0l/V3SXyR9MaX1BRpK8jSktLJ8mm9mxVHVp/m9JE0rWb8yIq6scN9DWLlXuhDYLCJekzQE+L2k7Wk6bEdLhTuY5qBf7/X4zXmH03vDdfk4gmtu/RuXjbufC045gH13G8SHHy3n+YbFjDn7Bt5a+h577LIN5528P6uv1oMPP1rGWf/ze/4y9RkADhqxE6cfszfdu3fjj3+dyfd/eUfBR2dNWb58ObvtOow+m2zC726/kxF7fImlS5cA8OqrixgydGfG33J7wa3sHKoMposjYmgr6ugBfB0Y0pgWER8AH6TPj0maC2xN1hPtV7J7P2BBS3U4mOZg2fKPOfPi25gxu4G11+rJQzedwX2PzOa+KbP5r0smsnz5x/zo5FGcdvQIfjD2Dl57cykHnfJrFr76Fttt2Yc7f3UiW+79Azb49Ke44JQD2PXQn7L4jaVcde5h7D5sa+5/9JmiD9FW8atLxzJw4Da8veRtAO79819WbDt09EF8db/9i2pap9NO95nuBcyOiBWn75I2Al6PiOWStgAGAM9FxOuSlkgaDjwCHA5c0lIFHjPNwcuL32bG7OxntPTdD5j9/MtsstF63DdlNsuXfwzAo08+T9/e6wHw+NMNLHw1u6D4j7kL6bn6aqy+Wg8277shz764iMVvLAXgz4/M5oA9m7rQaEV6qaGBe+6exBFHHfOJbUuWLOGB+/+P/fZv8XqFNcpxzFTSOOBhYKCkBkmNP6TRfPLC027AE5IeB34HHB8RjRevTgB+A8wB5tLClXxwzzR3m/XZgMED+zF15ryV0g8f9f/43b3TP5H/wL0G8/jT8/nwo2XMnf8qA/v3ZrM+G/DSojfZ/8s7sFqP7u3UcqvUGaf9B+ddcCFLlyz5xLY777idL315D9Zdd90CWtY55dkzjYhDmkk/som0W4Fbm8k/DRhUTd017ZlKGinp6XTj65m1rKsj+NSaqzPuomM57aJbWfLO+yvSTz9mb5Yv/5jxk6aulH/bLT7Dj04exUk/Gg/Am0ve4+QLbuaGnxzNfdf8By8seG1Fz9Y6hrsn3cVGG23MjjsNaXL77yaM5+BvjG7nVnVe1dwW1dEfO61Zz1RSd+Ay4CtkA7pTJU2MiH/Uqs4i9ejRjXEXHcfNd0/jjj8/viL90K/twr67DWKfb49dKX/fjdfj5ovHcOx/Xc/zDYtXpE96YCaTHsge3jj66593MO1gpjz0EJP+cCf3/vFu3v/gfZa8/TbHHnkYv7n2el577TWmTZvKTRNuK7qZnUq9TKhXy6MYBsyJiOci4kNgPDCqhvUV6oqzD+Xp519m7A1/XpH2lV235btH7sVBp/ya997/aEX6p9dek9suOZ7/vmQiDz/+3ErlbLT+2gCst86ajPnGF/nf2x9unwOwivzwRxfw9NwXmfXMc1x73U3stvuX+c211wPw+9tuYeQ+X2WNNdYouJWdTPvdZ1pTtRwz7QvML1lvAHZZNZOkMWSPbcFqa9ewObWz6+AtOHS/XXjymZeYMj4bzTj70on8/LSD6bl6D+66/CQAHn1yHiefP57jR+/GlptuxJnHjeTM40YC8LUTLuXVN5Zy0ekH8S9bZ/cH//jKPzLnxUXFHJRV7XcTJnDqaacX3YxOp6OfvldK2dNSNShYOhjYOyKOTeuHAcMi4jvN7dNtrY2j58Bv1KQ9VoxXp4xtOZN1GrvtOozpj03LLfr1/MyA6Hdo5f9Hnrt438dac59pe6hlz7QB2LRkvaIbX82s6xBQJx3Tmo6ZTgUGSNpc0upk93lNrGF9Ztbp+Gp+iyJimaSTgHuA7sA1ETGrVvWZWefUwWNkxWp6035ETCJ7F6CZWZM6eo+zUn4CysyKI/dMzczaTEC3OpkDysHUzArlYGpm1lY+zTcza7vsPtP6iKYOpmZWoI5//2ilHEzNrFB1EksdTM2sWO6Zmpm1VR1dgKqPt7KaWafUeAEqr2fzJV0jaZGkmSVp50h6SdKMtOxbsu17aSaQpyXtXZJe9SwhDqZmViip8qUC1wIjm0j/RUQMTsukrF5tR/YCpu3TPr+S1L1klpB9gO2AQ1Lesnyab2aFynlCvQck9a8w+yhgfER8ADwvaQ7ZDCGQZglJ7WucJaTslEvumZpZcZQ9AVXpAvSSNK1kGVNhTSdJeiINA6yf0pqaDaRvmfSyHEzNrDCNL4eu4jR/cUQMLVmurKCay4EtgcHAQuDnJdWvKsqkl+XTfDMrUO1v2o+IV1bUJl0F3JVWy80GUvUsIe6Zmlmhcr4A1UT56lOyeiDQeKV/IjBaUk9JmwMDgEdp5Swh7pmaWaHy7JlKGgfsTja22gCcDewuaTDZqfo84NsAETFL0gSyC0vLgBMjYnkqp+pZQhxMzaw4Od+0HxGHNJF8dZn85wPnN5Fe9SwhDqZmVhi/NcrMLCcOpmZmOaiTWOpgambFcs/UzKyt6uitUQ6mZlYYIU+oZ2aWh2510jV1MDWzQtVJLHUwNbPiZI+J1kc0dTA1s0LVyZCpg6mZFavue6aS1i23Y0S8nX9zzKyrqZNYWrZnOotPvii1cT2AzWrYLjPrAkR2e1Q9aDaYRsSmzW0zM8tLvYyZVvRyaEmjJZ2VPveTNKS2zTKzLqGKaZ47+thqi8FU0qXAl4HDUtK7wBW1bJSZdQ0CundTxUtHVsnV/F0jYidJfweIiNfTq/zNzNqsg3c4K1bJaf5HkrqRZueTtCHwcU1bZWZdRp6n+Wkq50WSZpak/UzS7DTV8+2S1kvp/SW9J2lGWq4o2WeIpCclzZE0VhVUXkkwvQy4FdhI0g+BB4GfVLCfmVlZ1UymV2EP9lpg5Cppk4FBEfE54BngeyXb5kbE4LQcX5J+OTCGbJK9AU2U+QktnuZHxHWSHgP2SkkHR8TMcvuYmVUqzxedRMQDkvqvknZvyeoU4KByZaTZTNeNiIfT+nXAAcDd5fardKrn7sBHwIdV7GNm1iJVsZDNOjqtZBlTZXVHs3JQ3FzS3yX9RdIXU1pfoKEkT0NKK6vFnqmk7wPfAm4nO56bJN0YET+utPVmZs2p8panxRExtJX1fJ9sSucbU9JCYLOIeC3d7vl7SdtDk08RREvlV3I1/1+BIRHxbmrQ+cBjgIOpmbWJaJ+b9iUdAewH7BkRARARHwAfpM+PSZoLbE3WE+1Xsns/YEFLdVRyyv4CKwfdHsBzlRyAmVlZ7XDTvqSRwBnA/o2dwpS+kaTu6fMWZBeanouIhcASScPTVfzDgTtaqqfci05+Qda1fReYJemetD6C7Iq+mVmb5XmfqaRxwO5kY6sNwNlkV+97ApNTQJ6SrtzvBpwraRmwHDg+Il5PRZ1AdmfAmmRjrGUvPkH50/zGK/azgD+UpE+p6KjMzFrQ+ARUXiLikCaSr24m761kt302tW0aMKiausu96KTJBpiZ5amjP3NfqUqu5m8JnA9sB6zRmB4RW9ewXWbWRdRHKK3sAtS1wP+SHfM+wARgfA3bZGZdhJTdtF/p0pFVEkzXioh7ACJibkT8gOwtUmZmbZbz46SFqeQ+0w/S7QFzJR0PvARsXNtmmVlX0WXGTIH/ANYGTiYbO/002SNZZmZtViextKIXnTySPi7hny+INjNrM9Hxx0IrVe6m/dsp8zxqRHy9Ji0ys66jE4yFVqpcz/TSdmtFsuO2m/G3R9q9Wquhjz9u8f0Q1sXV/ZhpRNzXng0xs66pXt7pWckFKDOzmsj7cdIiOZiaWaHqJJZWHkwl9Uzv/zMzy0V2M359RNMWhyskDZP0JPBsWt9B0iU1b5mZdQndVPnSkVUy9juW7A3VrwFExOP4cVIzy0lXepy0W0S8sEpXfHmN2mNmXUg2bUkHj5IVqiSYzpc0DIj0iv/vkM09bWbWZvVya1Qlx3ECcCqwGfAKMDylmZm1WZ6n+ZKukbRI0syStA0kTZb0bPq6fkqXpLGS5kh6QtJOJfsckfI/mybja1GLwTQiFkXE6IjolZbREbG4ksLNzMpRFe8yrXA44Fpg5CppZwL3RcQA4L60Dtn7mQekZQxweWrTBmRzR+0CDAPObgzA5VTypv2raOIZ/YgY09K+ZmYtyXPINCIekNR/leRRZJPsAfwWuJ9sttJRwHVp6ucpktaT1Cflndw4uZ6kyWQBely5uisZM/1Tyec1gAOB+RXsZ2ZWloAe1d3z1EvStJL1KyPiyhb26Z2mbyYiFkpqfB9zX1aOZQ0prbn0sip5Bd/NpeuSrgcmt7SfmVklquyZLo6IoXlV3URalEkvqzUX0jYHPtuK/czMVlbFDfttuGn/lXT6Tvq6KKU3AJuW5OsHLCiTXlYlT0C9Ien1tLxJ1is9q6JDMDNrgar410oTgcYr8kcAd5SkH56u6g8H3krDAfcAIyStny48jUhpZZU9zU9zP+1ANu8TwMdpsNbMrM2ym/ZzLE8aR3YBqZekBrKr8hcCEyQdA7wIHJyyTwL2BeYA7wJHAUTE65LOA6amfOc2Xowqp2wwjYiQdHtEDKn6qMzMKpBnMI2IQ5rZtGcTeQM4sZlyrgGuqabuSsZMHy29mdXMLE+SKl46snJzQPWIiGXAF4DjJM0F3iHrmUdEOMCaWZvkfZpfpHKn+Y8COwEHtFNbzKyr6QRvg6pUuWAqgIiY205tMbMuqCu8NWojSac2tzEiLq5Be8ysC8nmgCq6FfkoF0y7A2vT9NMAZmY5EN3qJMSUC6YLI+LcdmuJmXU5oguNmZqZ1UwnmNupUuWC6SducjUzy1vdX4Cq5PEpM7O26Cqn+WZmNVf3PVMzs/ZQJ7HUwdTMiiPqZ3ZSB1MzK47o8C8wqZSDqZkVqj5CqYOpmRVIQHf3TM3M2q5OYmndjP2aWadU+YuhKxlblTRQ0oyS5W1Jp0g6R9JLJen7luzzPUlzJD0tae/WHol7pmZWmLyv5kfE08BgAEndyeavu51sfqdfRMRFK9UvbQeMBrYHNgH+JGnriFhebd3umZpZoWo4bcmewNyIeKFMnlHA+Ij4ICKeJ5tcb1hrjsPB1MwKpSoWsllHp5UsY8oUPRoYV7J+kqQnJF2TpnAG6AvML8nTkNKq5mBqZsVR1T3TxRExtGS5sslipdWB/YFbUtLlwJZkQwALgZ//swWf0Krp7D1mamaFqeETUPsA0yPiFYDGrwCSrgLuSqsNwKYl+/UDFrSmQvdMzaxQNRozPYSSU3xJfUq2HQjMTJ8nAqMl9ZS0OTCAbDLRqrlnamaFyvs2U0lrAV8Bvl2S/FNJg8lO4ec1bouIWZImAP8AlgEntuZKPjiYmlmBavEEVES8C2y4StphZfKfD5zf1nodTM2sUPXyBJSDqZkVSKhOXnXiYGpmhXLP1MysjbJbo+ojmjqYmllx5J6pmVkuHEzNzHJQLxeg/ARUO7r3nj/yue0Hsv02W/Gzn15YdHOsFRrmz2efEXuw0+e2Y+jgQVx2yS8BOP+8c9hq834M33lHhu+8I3+8e1LBLe0cBHRT5UtH5p5pO1m+fDmnnHwif7h7Mn379eMLw3dmv/32Z9vttiu6aVaF7j16cMFPLmLHHXdiyZIlfGH4UPbY6ysAnPSdUzjl1P8suIWdT730TB1M28nURx9lyy23YvMttgDg4G+O5q4773Aw7WT69OlDnz7ZY97rrLMOA7fZlgUvvVRwqzq3bnUyaOrT/HayYMFL9Ov3z5fT9O3bj5f8S9ipvTBvHo8//nd2HrYLAL++4jKGDdmB48cczRtvvFFw6zqHejrNr1kwTS9gXSRpZsu561/EJ1+RWC/zhXdFS5cu5VujD+KnF/2Cddddl2PHnMDMp+YwZerf+cxn+vC9M75bdBM7CVX1ryOrZc/0WmBkDcvvVPr27UdDwz9f6P3SSw1ssskmBbbIWuujjz7iW988iG+O/hajDvg6AL1796Z79+5069aNo44+jmlTpxbcyk4i3Wda6dKR1SyYRsQDwOu1Kr+zGbrzzsyZ8yzznn+eDz/8kFtuHs9X99u/6GZZlSKCE759LAO32YaTTzl1RfrChQtXfJ54x+1sv/2gIprXKVU5bUmHVfgFqDSHyxiATTfbrODW1E6PHj34xS8v5Wtf3Zvly5dzxJFHs9322xfdLKvSww/9jXE3Xs/2g/6F4TvvCMA5557PLRPG88TjM5DEZz/bn7GXXVFwSzuHbMy0o4fJyhQeTNMcLlcCDBkytFVzr3QWI/fZl5H77NtyRuuwdv38F3jng48/ke6fa+vVRyjtAMHUzLq4OommvjXKzAqV99V8SfMkPSlphqRpKW0DSZMlPZu+rp/SJWmspDlpGuidWnsctbw1ahzwMDBQUoOkY2pVl5l1XjW6mv/liBgcEUPT+pnAfRExALgvrUM2i+mAtIwhmxK6VWp2mh8Rh9SqbDOrH+10lj8K2D19/i1wP3BGSr8ushvBp0haT1KfiFjYZCll+DTfzAojqp7quZekaSXLmCaKDeBeSY+VbO/dGCDT141Tel9gfsm+DSmtar4AZWbFqf70fXHJqXtzPh8RCyRtDEyWNLt8Cz6hVXcVuWdqZoXK+6b9iFiQvi4CbgeGAa9I6gOQvi5K2RuATUt27wcsaM1xOJiaWbFyjKaSPiVpncbPwAhgJjAROCJlOwK4I32eCByeruoPB95qzXgp+DTfzAqV+wtMegO3p/HVHsBNEfFHSVOBCemuoheBg1P+ScC+wBzgXeCo1lbsYGpmhcrzadKIeA7YoYn014A9m0gP4MQ86nYwNbPCdIYXmFTKwdTMilUn0dTB1MwK1dFf+lwpB1MzK1SdvIHPwdTMCtQJ3qBfKQdTMyuUT/PNzNooeza/6Fbkw8HUzApVJ7HUwdTMClYn0dTB1MwK5TFTM7MceMzUzCwHdRJLHUzNrGB1Ek0dTM2sMNmLTuojmjqYmllxBN3qI5Y6mJpZwRxMzczaKvc37RfGc0CZWaGkypeWy9Kmkv5P0lOSZkn695R+jqSXJM1Iy74l+3xP0hxJT0vau7XH4Z6pmRWmBm/aXwZ8NyKmp4n1HpM0OW37RURctFL90nbAaGB7YBPgT5K2jojl1VbsnqmZFSvH2UkjYmFETE+flwBPAX3L7DIKGB8RH0TE82QT6w1rzWE4mJpZoVTFv6rKlfoDOwKPpKSTJD0h6RpJ66e0vsD8kt0aKB98m+VgamaFqnLMtJekaSXLmKbL1NrArcApEfE2cDmwJTAYWAj8vDFrE7tHa47DY6ZmVqgqx0wXR8TQsuVJq5EF0hsj4jaAiHilZPtVwF1ptQHYtGT3fsCC6pqUcc/UzIpTRa+0wqv5Aq4GnoqIi0vS+5RkOxCYmT5PBEZL6ilpc2AA8GhrDsU9UzMrWK7X8z8PHAY8KWlGSjsLOETSYLJT+HnAtwEiYpakCcA/yO4EOLE1V/LBwdTMCiTyfZw0Ih6k6eg8qcw+5wPnt7VuB1MzK5TfZ2pmloN6eZzUwdTMilUfsdTB1MyKVSex1MHUzIpT6S1PnYGDqZkVymOmZmZ5qI9Y6mBqZsWqk1jqYGpmxfKYqZlZGwnRrU6iqV90YmaWA/dMzaxQddIxdTA1s2L51igzs7byTftmZm1Xg9lJC+NgambFqpNo6mBqZoXQX9aGAAAE2ElEQVTymKmZWQ7qZczU95maWaFUxVJRedJISU9LmiPpzBo0uUnumZpZoZRj11RSd+Ay4Ctk0zhPlTQxIv6RWyXNcM/UzAoj8p3qGRgGzImI5yLiQ2A8MKqGh7BCh+qZTp/+2OI1V9MLRbejHfQCFhfdCMtVV/mZfjbPwqZPf+yeNVdTryp2WUPStJL1KyPiypL1vsD8kvUGYJe2tLFSHSqYRsRGRbehPUiaFhFDi26H5cc/09aJiJE5F9lU/zVyrqNJPs03s3rSAGxast4PWNAeFTuYmlk9mQoMkLS5pNWB0cDE9qi4Q53mdyFXtpzFOhn/TDuAiFgm6STgHqA7cE1EzGqPuhXRLsMJZmZ1zaf5ZmY5cDA1M8uBg2k7KuoxN6sdSddIWiRpZtFtsWI5mLaTksfc9gG2Aw6RtF2xrbIcXAvkfa+kdUIOpu2nsMfcrHYi4gHg9aLbYcVzMG0/TT3m1regtphZzhxM209hj7mZWe05mLafwh5zM7PaczBtP4U95mZmtedg2k4iYhnQ+JjbU8CE9nrMzWpH0jjgYWCgpAZJxxTdJiuGHyc1M8uBe6ZmZjlwMDUzy4GDqZlZDhxMzcxy4GBqZpYDB9M6Imm5pBmSZkq6RdJabShrd0l3pc/7l3vLlaT1JP1bK+o4R9J/Vpq+Sp5rJR1URV39/WYnqyUH0/ryXkQMjohBwIfA8aUblan6Zx4REyPiwjJZ1gOqDqZm9cTBtH79Fdgq9ciekvQrYDqwqaQRkh6WND31YNeGFe9bnS3pQeDrjQVJOlLSpelzb0m3S3o8LbsCFwJbpl7xz1K+0yRNlfSEpB+WlPX99E7XPwEDWzoIScelch6XdOsqve29JP1V0jOS9kv5u0v6WUnd327rN9KsEg6mdUhSD7L3pj6ZkgYC10XEjsA7wA+AvSJiJ2AacKqkNYCrgK8BXwQ+00zxY4G/RMQOwE7ALOBMYG7qFZ8maQQwgOy1g4OBIZJ2kzSE7DHaHcmC9c4VHM5tEbFzqu8poPQJo/7Al4CvAlekYzgGeCsidk7lHydp8wrqMWsTz05aX9aUNCN9/itwNbAJ8EJETEnpw8leTv03SQCrkz0OuQ3wfEQ8CyDpBmBME3XsARwOEBHLgbckrb9KnhFp+XtaX5ssuK4D3B4R76Y6Knk3wSBJPyIbSlib7HHcRhMi4mPgWUnPpWMYAXyuZDz106nuZyqoy6zVHEzry3sRMbg0IQXMd0qTgMkRccgq+QaT3ysBBfw4In69Sh2ntKKOa4EDIuJxSUcCu5dsW7WsSHV/JyJKgy6S+ldZr1lVfJrf9UwBPi9pKwBJa0naGpgNbC5py5TvkGb2vw84Ie3bXdK6wBKyXmeje4CjS8Zi+0raGHgAOFDSmpLWIRtSaMk6wEJJqwGHrrLtYEndUpu3AJ5OdZ+Q8iNpa0mfqqAeszZxz7SLiYhXUw9vnKSeKfkHEfGMpDHAHyQtBh4EBjVRxL8DV6a3Iy0HToiIhyX9Ld16dHcaN90WeDj1jJcC/xoR0yXdDMwAXiAbimjJfwGPpPxPsnLQfhr4C9AbOD4i3pf0G7Kx1OnKKn8VOKCy745Z6/mtUWZmOfBpvplZDhxMzcxy4GBqZpYDB1Mzsxw4mJqZ5cDB1MwsBw6mZmY5+P/+KsLEveNNmAAAAABJRU5ErkJggg==\n",
      "text/plain": [
       "<Figure size 432x288 with 2 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "#cnf_matrix_tra = confusion_matrix(Y_test, predictions)\n",
    "\n",
    "print(\"Recall metric in the train dataset: {}%\".format(100*cnf_matrix_tra[1,1]/(cnf_matrix_tra[1,0]+cnf_matrix_tra[1,1])))\n",
    "\n",
    "\n",
    "class_names = [0,1]\n",
    "plt.figure()\n",
    "plot_confusion_matrix(cnf_matrix_tra , classes=class_names, title='Confusion matrix of LDA')\n",
    "plt.savefig('static/img/modelComparison/flood_confusionmatrix.png')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.4"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}

NameError: name 'null' is not defined